# Ordering by metrics and retraining phase

## Dataset: Intel

## Experiment configuration 1


## Configuration 1
	1. Incremental guided retraining starting from scratch using the new adversarial inputs and original training set

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
import gc

In [3]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [4]:
cd C:/Users/fjdur/Documents/utils/

[WinError 3] The system cannot find the path specified: 'C:/Users/fjdur/Documents/utils/'
C:\Users\fjdurlop\Documents\upc\upc-july\notebooks


In [5]:
cd C:/Users/fjdurlop/Documents/upc/utils/

C:\Users\fjdurlop\Documents\upc\utils


In [6]:
# utils for project
#import utils_guided_retraining as utils
import utils_guided_retraining2 as utils

2.5.0
keras
2.5.0


In [7]:
#cd '../notebooks/'

In [8]:
dataset = "cifar"

In [9]:
x_train,y_train = utils.get_data(dataset,"Train", verbose=True)
x_val,y_val = utils.get_data(dataset,"Val", verbose=True)
x_test,y_test = utils.get_data(dataset,"Test", verbose=True)

C:/Users/fjdurlop/Documents/upc/train_data/cifar/x_train.npy
x_set len:  41999
C:/Users/fjdurlop/Documents/upc/train_data/cifar/y_train.npy
y_set len:  41999
C:/Users/fjdurlop/Documents/upc/train_data/cifar/x_val.npy
x_set len:  12000
C:/Users/fjdurlop/Documents/upc/train_data/cifar/y_val.npy
y_set len:  12000
C:/Users/fjdurlop/Documents/upc/train_data/cifar/x_test.npy
x_set len:  6001
C:/Users/fjdurlop/Documents/upc/train_data/cifar/y_test.npy
y_set len:  6001


In [10]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary")

In [11]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6000
6000


In [12]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [13]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

6000
6000


## ----

In [ ]:
model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02"

In [ ]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"


In [14]:
x_train,y_train = utils.get_data(dataset,"Train", verbose=True)
x_val,y_val = utils.get_data(dataset,"Val", verbose=True)
x_test,y_test = utils.get_data(dataset,"Test", verbose=True)

In [15]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
Model loaded correctly


In [16]:
print(x_train_and_adversary.shape)
len(x_train_and_adversary)//20

(47999, 32, 32, 1)


2399

In [17]:
n_data_points = 2400

In [18]:
import gc

## Loading LSA and DSA values 

In [16]:
#lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"
lsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/lsa_values.npy"


In [17]:
lsa_values = np.load(lsa_direction)


In [18]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_train_and_adversary)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_train_and_adversary)


In [19]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [20]:
len(image_sets_lsa)

20

## Training guided by LSA

In [21]:
len(label_sets_lsa)

20

In [22]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [23]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n=n+1

Epoch 1/20
38/38 [==============================] - 20s 171ms/step - loss: 2.8258 - accuracy: 0.2099 - val_loss: 1.6662 - val_accuracy: 0.4126
Epoch 2/20
38/38 [==============================] - 6s 160ms/step - loss: 1.7812 - accuracy: 0.3248 - val_loss: 1.4367 - val_accuracy: 0.4859
Epoch 3/20
38/38 [==============================] - 6s 164ms/step - loss: 1.6508 - accuracy: 0.3920 - val_loss: 1.4531 - val_accuracy: 0.4952
Epoch 4/20
38/38 [==============================] - 6s 164ms/step - loss: 1.4916 - accuracy: 0.4550 - val_loss: 1.4121 - val_accuracy: 0.4933
Epoch 5/20
38/38 [==============================] - 7s 184ms/step - loss: 1.3919 - accuracy: 0.4942 - val_loss: 1.2959 - val_accuracy: 0.5642
Epoch 6/20
38/38 [==============================] - 7s 200ms/step - loss: 1.2698 - accuracy: 0.5246 - val_loss: 1.2827 - val_accuracy: 0.5470
Epoch 7/20
38/38 [==============================] - 8s 207ms/step - loss: 1.1683 - accuracy: 0.5703 - val_loss: 1.4383 - val_accuracy: 0.5175
Epoch

In [24]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

Epoch 1/20
75/75 [==============================] - 16s 185ms/step - loss: 1.5962 - accuracy: 0.4891 - val_loss: 1.2934 - val_accuracy: 0.6126
Epoch 2/20
75/75 [==============================] - 14s 186ms/step - loss: 1.2065 - accuracy: 0.5757 - val_loss: 1.1715 - val_accuracy: 0.6114
Epoch 3/20
75/75 [==============================] - 13s 178ms/step - loss: 1.1389 - accuracy: 0.5961 - val_loss: 1.2207 - val_accuracy: 0.5549
Epoch 4/20
75/75 [==============================] - 13s 177ms/step - loss: 1.1061 - accuracy: 0.6098 - val_loss: 1.1428 - val_accuracy: 0.5685
Epoch 5/20
75/75 [==============================] - 14s 186ms/step - loss: 0.9857 - accuracy: 0.6414 - val_loss: 1.1502 - val_accuracy: 0.5818
Epoch 6/20
75/75 [==============================] - 16s 213ms/step - loss: 0.9570 - accuracy: 0.6398 - val_loss: 1.0397 - val_accuracy: 0.6162
Epoch 7/20
75/75 [==============================] - 14s 181ms/step - loss: 0.9196 - accuracy: 0.6650 - val_loss: 1.1657 - val_accuracy: 0.5702

In [25]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 20s 162ms/step - loss: 1.2105 - accuracy: 0.6267 - val_loss: 1.2071 - val_accuracy: 0.5810
Epoch 2/20
113/113 [==============================] - 18s 158ms/step - loss: 0.9225 - accuracy: 0.6822 - val_loss: 1.2059 - val_accuracy: 0.5716
Epoch 3/20
113/113 [==============================] - 18s 159ms/step - loss: 0.9019 - accuracy: 0.6871 - val_loss: 1.0361 - val_accuracy: 0.6298
Epoch 4/20
113/113 [==============================] - 18s 159ms/step - loss: 0.8241 - accuracy: 0.7167 - val_loss: 1.1225 - val_accuracy: 0.5998
Epoch 5/20
113/113 [==============================] - 18s 159ms/step - loss: 0.7631 - accuracy: 0.7238 - val_loss: 1.3779 - val_accuracy: 0.5269
Epoch 6/20
113/113 [==============================] - 18s 158ms/step - loss: 0.7697 - accuracy: 0.7297 - val_loss: 1.1811 - val_accuracy: 0.5608
Epoch 7/20
113/113 [==============================] - 18s 158ms/step - loss: 0.7227 - accuracy: 0.7483 - val_loss: 1.0897 - val_

In [26]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 24s 149ms/step - loss: 1.0323 - accuracy: 0.6951 - val_loss: 1.0227 - val_accuracy: 0.6559
Epoch 2/20
150/150 [==============================] - 22s 144ms/step - loss: 0.7741 - accuracy: 0.7391 - val_loss: 0.9697 - val_accuracy: 0.6797
Epoch 3/20
150/150 [==============================] - 21s 143ms/step - loss: 0.7280 - accuracy: 0.7529 - val_loss: 1.0247 - val_accuracy: 0.6351
Epoch 4/20
150/150 [==============================] - 21s 143ms/step - loss: 0.7066 - accuracy: 0.7526 - val_loss: 1.1121 - val_accuracy: 0.5932
Epoch 5/20
150/150 [==============================] - 21s 143ms/step - loss: 0.6746 - accuracy: 0.7696 - val_loss: 0.9437 - val_accuracy: 0.6622
Epoch 6/20
150/150 [==============================] - 22s 149ms/step - loss: 0.6460 - accuracy: 0.7749 - val_loss: 1.0295 - val_accuracy: 0.6249
Epoch 7/20
150/150 [==============================] - 22s 147ms/step - loss: 0.6195 - accuracy: 0.7802 - val_loss: 1.0840 - val_

In [27]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 29s 143ms/step - loss: 0.8793 - accuracy: 0.7210 - val_loss: 0.9309 - val_accuracy: 0.7131
Epoch 2/20
188/188 [==============================] - 27s 145ms/step - loss: 0.7584 - accuracy: 0.7417 - val_loss: 0.9803 - val_accuracy: 0.6905
Epoch 3/20
188/188 [==============================] - 28s 150ms/step - loss: 0.6780 - accuracy: 0.7632 - val_loss: 1.0645 - val_accuracy: 0.6863
Epoch 4/20
188/188 [==============================] - 25s 134ms/step - loss: 0.6472 - accuracy: 0.7741 - val_loss: 0.9708 - val_accuracy: 0.7032
Epoch 5/20
188/188 [==============================] - 25s 133ms/step - loss: 0.6369 - accuracy: 0.7813 - val_loss: 1.0770 - val_accuracy: 0.6492
Epoch 6/20
188/188 [==============================] - 25s 131ms/step - loss: 0.5938 - accuracy: 0.7902 - val_loss: 1.1565 - val_accuracy: 0.6224
Epoch 7/20
188/188 [==============================] - 25s 132ms/step - loss: 0.5790 - accuracy: 0.7978 - val_loss: 0.9360 - val_

In [28]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 30s 125ms/step - loss: 0.8785 - accuracy: 0.7246 - val_loss: 0.9510 - val_accuracy: 0.7202
Epoch 2/20
225/225 [==============================] - 28s 124ms/step - loss: 0.7049 - accuracy: 0.7558 - val_loss: 0.9148 - val_accuracy: 0.7043
Epoch 3/20
225/225 [==============================] - 28s 124ms/step - loss: 0.6564 - accuracy: 0.7690 - val_loss: 0.9644 - val_accuracy: 0.7178
Epoch 4/20
225/225 [==============================] - 28s 124ms/step - loss: 0.6156 - accuracy: 0.7863 - val_loss: 1.0204 - val_accuracy: 0.6822
Epoch 5/20
225/225 [==============================] - 28s 125ms/step - loss: 0.5892 - accuracy: 0.7970 - val_loss: 1.0577 - val_accuracy: 0.6643
Epoch 6/20
225/225 [==============================] - 28s 124ms/step - loss: 0.5863 - accuracy: 0.7946 - val_loss: 0.8838 - val_accuracy: 0.7075
Epoch 7/20
225/225 [==============================] - 28s 124ms/step - loss: 0.5497 - accuracy: 0.8094 - val_loss: 0.9573 - val_

In [29]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 34s 123ms/step - loss: 0.8563 - accuracy: 0.7360 - val_loss: 0.9064 - val_accuracy: 0.7130
Epoch 2/20
263/263 [==============================] - 32s 122ms/step - loss: 0.6852 - accuracy: 0.7680 - val_loss: 0.9850 - val_accuracy: 0.6767
Epoch 3/20
263/263 [==============================] - 32s 122ms/step - loss: 0.6569 - accuracy: 0.7759 - val_loss: 0.9715 - val_accuracy: 0.7068
Epoch 4/20
263/263 [==============================] - 32s 122ms/step - loss: 0.6134 - accuracy: 0.7879 - val_loss: 1.0622 - val_accuracy: 0.6777
Epoch 5/20
263/263 [==============================] - 33s 124ms/step - loss: 0.5899 - accuracy: 0.8018 - val_loss: 1.1671 - val_accuracy: 0.6122
Epoch 6/20
263/263 [==============================] - 32s 122ms/step - loss: 0.5727 - accuracy: 0.8010 - val_loss: 1.0145 - val_accuracy: 0.6879
Epoch 7/20
263/263 [==============================] - 32s 122ms/step - loss: 0.5573 - accuracy: 0.8084 - val_loss: 1.0836 - val_

In [30]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 40s 127ms/step - loss: 0.8428 - accuracy: 0.7344 - val_loss: 1.0322 - val_accuracy: 0.6979
Epoch 2/20
300/300 [==============================] - 37s 125ms/step - loss: 0.7107 - accuracy: 0.7620 - val_loss: 0.9136 - val_accuracy: 0.7160
Epoch 3/20
300/300 [==============================] - 37s 124ms/step - loss: 0.6783 - accuracy: 0.7667 - val_loss: 0.9297 - val_accuracy: 0.7167
Epoch 4/20
300/300 [==============================] - 37s 124ms/step - loss: 0.6448 - accuracy: 0.7823 - val_loss: 0.9004 - val_accuracy: 0.6984
Epoch 5/20
300/300 [==============================] - 37s 124ms/step - loss: 0.6282 - accuracy: 0.7869 - val_loss: 0.9128 - val_accuracy: 0.7014
Epoch 6/20
300/300 [==============================] - 37s 124ms/step - loss: 0.6164 - accuracy: 0.7885 - val_loss: 0.8950 - val_accuracy: 0.7043
Epoch 7/20
300/300 [==============================] - 37s 124ms/step - loss: 0.6109 - accuracy: 0.7918 - val_loss: 0.9294 - val_

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 44s 126ms/step - loss: 0.8801 - accuracy: 0.7194 - val_loss: 1.2555 - val_accuracy: 0.6386
Epoch 2/20
338/338 [==============================] - 42s 124ms/step - loss: 0.7582 - accuracy: 0.7451 - val_loss: 0.9401 - val_accuracy: 0.6667
Epoch 3/20
338/338 [==============================] - 43s 126ms/step - loss: 0.7175 - accuracy: 0.7582 - val_loss: 1.0344 - val_accuracy: 0.6771
Epoch 4/20
338/338 [==============================] - 44s 130ms/step - loss: 0.7017 - accuracy: 0.7634 - val_loss: 0.8116 - val_accuracy: 0.7218
Epoch 5/20
338/338 [==============================] - 42s 124ms/step - loss: 0.6870 - accuracy: 0.7680 - val_loss: 0.9362 - val_accuracy: 0.6985
Epoch 6/20
338/338 [==============================] - 42s 124ms/step - loss: 0.6605 - accuracy: 0.7744 - val_loss: 1.0208 - val_accuracy: 0.6385
Epoch 7/20
338/338 [==============================] - 42s 123ms/step - loss: 0.6399 - accuracy: 0.7806 - val_loss: 0.8392 - val_

In [32]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 48s 122ms/step - loss: 0.8812 - accuracy: 0.7154 - val_loss: 0.9228 - val_accuracy: 0.7236
Epoch 2/20
375/375 [==============================] - 45s 121ms/step - loss: 0.7582 - accuracy: 0.7413 - val_loss: 0.9222 - val_accuracy: 0.7126
Epoch 3/20
375/375 [==============================] - 45s 121ms/step - loss: 0.7188 - accuracy: 0.7616 - val_loss: 0.8458 - val_accuracy: 0.7530
Epoch 4/20
375/375 [==============================] - 45s 121ms/step - loss: 0.6980 - accuracy: 0.7629 - val_loss: 0.9314 - val_accuracy: 0.6952
Epoch 5/20
375/375 [==============================] - 45s 121ms/step - loss: 0.6843 - accuracy: 0.7652 - val_loss: 0.8479 - val_accuracy: 0.6997
Epoch 6/20
375/375 [==============================] - 45s 121ms/step - loss: 0.6731 - accuracy: 0.7700 - val_loss: 0.9214 - val_accuracy: 0.7077
Epoch 7/20
375/375 [==============================] - 46s 122ms/step - loss: 0.6509 - accuracy: 0.7765 - val_loss: 0.8691 - val_

In [33]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 52s 123ms/step - loss: 0.8763 - accuracy: 0.7159 - val_loss: 0.8490 - val_accuracy: 0.7399
Epoch 2/20
413/413 [==============================] - 50s 121ms/step - loss: 0.7729 - accuracy: 0.7426 - val_loss: 0.8872 - val_accuracy: 0.7371
Epoch 3/20
413/413 [==============================] - 51s 122ms/step - loss: 0.7431 - accuracy: 0.7464 - val_loss: 0.8427 - val_accuracy: 0.7307
Epoch 4/20
413/413 [==============================] - 51s 122ms/step - loss: 0.7084 - accuracy: 0.7595 - val_loss: 0.9456 - val_accuracy: 0.6756
Epoch 5/20
413/413 [==============================] - 54s 131ms/step - loss: 0.6972 - accuracy: 0.7638 - val_loss: 0.7975 - val_accuracy: 0.7431
Epoch 6/20
413/413 [==============================] - 55s 133ms/step - loss: 0.7002 - accuracy: 0.7632 - val_loss: 1.0485 - val_accuracy: 0.6649
Epoch 7/20
413/413 [==============================] - 53s 128ms/step - loss: 0.6727 - accuracy: 0.7703 - val_loss: 0.8249 - val

In [34]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 56s 121ms/step - loss: 0.8856 - accuracy: 0.7135 - val_loss: 0.8891 - val_accuracy: 0.7616
Epoch 2/20
450/450 [==============================] - 55s 122ms/step - loss: 0.7675 - accuracy: 0.7382 - val_loss: 0.8265 - val_accuracy: 0.7372
Epoch 3/20
450/450 [==============================] - 54s 120ms/step - loss: 0.7363 - accuracy: 0.7519 - val_loss: 0.8656 - val_accuracy: 0.7312
Epoch 4/20
450/450 [==============================] - 54s 120ms/step - loss: 0.7242 - accuracy: 0.7545 - val_loss: 0.8718 - val_accuracy: 0.7363
Epoch 5/20
450/450 [==============================] - 54s 120ms/step - loss: 0.6977 - accuracy: 0.7628 - val_loss: 0.9778 - val_accuracy: 0.6934
Epoch 6/20
450/450 [==============================] - 54s 120ms/step - loss: 0.6918 - accuracy: 0.7620 - val_loss: 0.8755 - val_accuracy: 0.7050
Epoch 7/20
450/450 [==============================] - 54s 120ms/step - loss: 0.6787 - accuracy: 0.7693 - val_loss: 0.9063 - val

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 62s 122ms/step - loss: 0.8549 - accuracy: 0.7234 - val_loss: 0.7721 - val_accuracy: 0.7843
Epoch 2/20
488/488 [==============================] - 59s 121ms/step - loss: 0.7700 - accuracy: 0.7421 - val_loss: 0.8049 - val_accuracy: 0.7101
Epoch 3/20
488/488 [==============================] - 59s 122ms/step - loss: 0.7244 - accuracy: 0.7533 - val_loss: 0.7453 - val_accuracy: 0.7594
Epoch 4/20
488/488 [==============================] - 59s 122ms/step - loss: 0.7106 - accuracy: 0.7599 - val_loss: 0.7624 - val_accuracy: 0.7253
Epoch 5/20
488/488 [==============================] - 60s 123ms/step - loss: 0.7161 - accuracy: 0.7633 - val_loss: 0.8155 - val_accuracy: 0.7129
Epoch 6/20
488/488 [==============================] - 60s 124ms/step - loss: 0.6942 - accuracy: 0.7665 - val_loss: 0.7915 - val_accuracy: 0.7261
Epoch 7/20
488/488 [==============================] - 62s 128ms/step - loss: 0.6877 - accuracy: 0.7708 - val_loss: 0.8438 - val

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 67s 123ms/step - loss: 0.8384 - accuracy: 0.7297 - val_loss: 0.8087 - val_accuracy: 0.7576
Epoch 2/20
525/525 [==============================] - 61s 117ms/step - loss: 0.7477 - accuracy: 0.7516 - val_loss: 0.8235 - val_accuracy: 0.7096
Epoch 3/20
525/525 [==============================] - 61s 117ms/step - loss: 0.7338 - accuracy: 0.7526 - val_loss: 0.7870 - val_accuracy: 0.7447
Epoch 4/20
525/525 [==============================] - 61s 117ms/step - loss: 0.7072 - accuracy: 0.7606 - val_loss: 0.7984 - val_accuracy: 0.7203
Epoch 5/20
525/525 [==============================] - 62s 118ms/step - loss: 0.6966 - accuracy: 0.7655 - val_loss: 0.7434 - val_accuracy: 0.7297
Epoch 6/20
525/525 [==============================] - 61s 117ms/step - loss: 0.6882 - accuracy: 0.7680 - val_loss: 0.7531 - val_accuracy: 0.7196
Epoch 7/20
525/525 [==============================] - 62s 118ms/step - loss: 0.6946 - accuracy: 0.7694 - val_loss: 0.8319 - val

In [37]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 68s 118ms/step - loss: 0.8595 - accuracy: 0.7162 - val_loss: 0.8675 - val_accuracy: 0.7450
Epoch 2/20
563/563 [==============================] - 66s 117ms/step - loss: 0.7777 - accuracy: 0.7350 - val_loss: 0.7343 - val_accuracy: 0.7623
Epoch 3/20
563/563 [==============================] - 66s 117ms/step - loss: 0.7486 - accuracy: 0.7484 - val_loss: 0.8766 - val_accuracy: 0.6957
Epoch 4/20
563/563 [==============================] - 66s 117ms/step - loss: 0.7394 - accuracy: 0.7494 - val_loss: 0.7602 - val_accuracy: 0.7283
Epoch 5/20
563/563 [==============================] - 66s 117ms/step - loss: 0.6995 - accuracy: 0.7651 - val_loss: 0.8581 - val_accuracy: 0.7140
Epoch 6/20
563/563 [==============================] - 66s 117ms/step - loss: 0.7047 - accuracy: 0.7590 - val_loss: 0.8446 - val_accuracy: 0.6990
Epoch 7/20
563/563 [==============================] - 66s 117ms/step - loss: 0.7065 - accuracy: 0.7622 - val_loss: 0.8483 - val

In [38]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 77s 122ms/step - loss: 0.8692 - accuracy: 0.7163 - val_loss: 0.6913 - val_accuracy: 0.7813
Epoch 2/20
600/600 [==============================] - 71s 119ms/step - loss: 0.7896 - accuracy: 0.7298 - val_loss: 0.7596 - val_accuracy: 0.7781
Epoch 3/20
600/600 [==============================] - 71s 119ms/step - loss: 0.7644 - accuracy: 0.7396 - val_loss: 0.7927 - val_accuracy: 0.7400
Epoch 4/20
600/600 [==============================] - 71s 119ms/step - loss: 0.7517 - accuracy: 0.7421 - val_loss: 0.6613 - val_accuracy: 0.7785
Epoch 5/20
600/600 [==============================] - 72s 119ms/step - loss: 0.7437 - accuracy: 0.7448 - val_loss: 0.7706 - val_accuracy: 0.7572
Epoch 6/20
600/600 [==============================] - 71s 118ms/step - loss: 0.7218 - accuracy: 0.7545 - val_loss: 0.8496 - val_accuracy: 0.7287
Epoch 7/20
600/600 [==============================] - 72s 119ms/step - loss: 0.7209 - accuracy: 0.7527 - val_loss: 0.7290 - val

In [39]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 80s 122ms/step - loss: 0.8869 - accuracy: 0.7001 - val_loss: 0.7388 - val_accuracy: 0.7782
Epoch 2/20
638/638 [==============================] - 76s 120ms/step - loss: 0.8109 - accuracy: 0.7200 - val_loss: 0.7306 - val_accuracy: 0.7534
Epoch 3/20
638/638 [==============================] - 76s 120ms/step - loss: 0.7817 - accuracy: 0.7285 - val_loss: 0.7128 - val_accuracy: 0.7753
Epoch 4/20
638/638 [==============================] - 76s 120ms/step - loss: 0.7613 - accuracy: 0.7380 - val_loss: 0.7449 - val_accuracy: 0.7791
Epoch 5/20
638/638 [==============================] - 77s 121ms/step - loss: 0.7510 - accuracy: 0.7413 - val_loss: 0.8353 - val_accuracy: 0.7329
Epoch 6/20
638/638 [==============================] - 77s 120ms/step - loss: 0.7556 - accuracy: 0.7449 - val_loss: 0.8108 - val_accuracy: 0.7372
Epoch 7/20
638/638 [==============================] - 77s 121ms/step - loss: 0.7439 - accuracy: 0.7452 - val_loss: 0.7292 - val

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 83s 120ms/step - loss: 0.8792 - accuracy: 0.7062 - val_loss: 0.6248 - val_accuracy: 0.8108
Epoch 2/20
675/675 [==============================] - 81s 120ms/step - loss: 0.8224 - accuracy: 0.7157 - val_loss: 0.7042 - val_accuracy: 0.7894
Epoch 3/20
675/675 [==============================] - 82s 121ms/step - loss: 0.7919 - accuracy: 0.7290 - val_loss: 0.7368 - val_accuracy: 0.7561
Epoch 4/20
675/675 [==============================] - 81s 120ms/step - loss: 0.7742 - accuracy: 0.7338 - val_loss: 0.7593 - val_accuracy: 0.7724
Epoch 5/20
675/675 [==============================] - 80s 119ms/step - loss: 0.7644 - accuracy: 0.7401 - val_loss: 0.6670 - val_accuracy: 0.7747
Epoch 6/20
675/675 [==============================] - 80s 119ms/step - loss: 0.7641 - accuracy: 0.7381 - val_loss: 0.7340 - val_accuracy: 0.7561
Epoch 7/20
675/675 [==============================] - 80s 119ms/step - loss: 0.7490 - accuracy: 0.7445 - val_loss: 0.6628 - val

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 89s 122ms/step - loss: 0.8983 - accuracy: 0.6980 - val_loss: 0.6626 - val_accuracy: 0.8037
Epoch 2/20
713/713 [==============================] - 86s 120ms/step - loss: 0.8306 - accuracy: 0.7114 - val_loss: 0.6820 - val_accuracy: 0.8008
Epoch 3/20
713/713 [==============================] - 86s 121ms/step - loss: 0.8104 - accuracy: 0.7211 - val_loss: 0.8891 - val_accuracy: 0.7376
Epoch 4/20
713/713 [==============================] - 86s 121ms/step - loss: 0.7977 - accuracy: 0.7259 - val_loss: 0.6865 - val_accuracy: 0.7988
Epoch 5/20
713/713 [==============================] - 86s 121ms/step - loss: 0.7789 - accuracy: 0.7322 - val_loss: 0.6311 - val_accuracy: 0.8036
Epoch 6/20
713/713 [==============================] - 87s 121ms/step - loss: 0.7797 - accuracy: 0.7312 - val_loss: 0.6032 - val_accuracy: 0.8101
Epoch 7/20
713/713 [==============================] - 86s 121ms/step - loss: 0.7864 - accuracy: 0.7280 - val_loss: 0.6252 - val

In [42]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 93s 121ms/step - loss: 0.8845 - accuracy: 0.7066 - val_loss: 0.6737 - val_accuracy: 0.7945
Epoch 2/20
750/750 [==============================] - 90s 120ms/step - loss: 0.8186 - accuracy: 0.7190 - val_loss: 0.5274 - val_accuracy: 0.8297
Epoch 3/20
750/750 [==============================] - 90s 120ms/step - loss: 0.7986 - accuracy: 0.7297 - val_loss: 0.6430 - val_accuracy: 0.8034
Epoch 4/20
750/750 [==============================] - 91s 121ms/step - loss: 0.7900 - accuracy: 0.7298 - val_loss: 0.6018 - val_accuracy: 0.8042
Epoch 5/20
750/750 [==============================] - 90s 120ms/step - loss: 0.7715 - accuracy: 0.7350 - val_loss: 0.6618 - val_accuracy: 0.7918
Epoch 6/20
750/750 [==============================] - 91s 121ms/step - loss: 0.7669 - accuracy: 0.7363 - val_loss: 0.5815 - val_accuracy: 0.8168
Epoch 7/20
750/750 [==============================] - 90s 120ms/step - loss: 0.7695 - accuracy: 0.7354 - val_loss: 0.6000 - val

In [ ]:
time.sleep(400)

In [ ]:
len(models_lsa)

In [43]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
#new_model_lsa_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_lsa_e1"
new_model_lsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_lsa_e1"


i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_lsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_lsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_lsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_lsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_lsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_lsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_lsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets writ

In [44]:
import gc

In [45]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [46]:
gc.collect()

492698

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        #model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model(dataset,True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [47]:
#dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"
dsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/dsa_values.npy"

In [48]:
dsa_values = np.load(dsa_direction)

In [49]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_train_and_adversary)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_train_and_adversary)

In [50]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [51]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [52]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 13s 298ms/step - loss: 2.8043 - accuracy: 0.1588 - val_loss: 1.4624 - val_accuracy: 0.5244
Epoch 2/20
38/38 [==============================] - 10s 275ms/step - loss: 1.8660 - accuracy: 0.2982 - val_loss: 1.5164 - val_accuracy: 0.5130
Epoch 3/20
38/38 [==============================] - 10s 274ms/step - loss: 1.6764 - accuracy: 0.3792 - val_loss: 1.4566 - val_accuracy: 0.5142
Epoch 4/20
38/38 [==============================] - 10s 272ms/step - loss: 1.5385 - accuracy: 0.4325 - val_loss: 1.3701 - val_accuracy: 0.5409
Epoch 5/20
38/38 [==============================] - 10s 272ms/step - loss: 1.4472 - accuracy: 0.4871 - val_loss: 1.2662 - val_accuracy: 0.5670
Epoch 6/20
38/38 [==============================] - 10s 271ms/step - loss: 1.3103 - accuracy: 0.5313 - val_loss: 1.1298 - val_accuracy: 0.6127
Epoch 7/20
38/38 [==============================] - 10s 274ms/step - loss: 1.1972 - accuracy: 0.5562 - val_loss: 1.1838 - val_accuracy: 0.59

In [53]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 17s 202ms/step - loss: 2.3821 - accuracy: 0.2269 - val_loss: 1.3547 - val_accuracy: 0.5620
Epoch 2/20
75/75 [==============================] - 14s 193ms/step - loss: 1.7290 - accuracy: 0.3689 - val_loss: 1.1579 - val_accuracy: 0.6026
Epoch 3/20
75/75 [==============================] - 14s 194ms/step - loss: 1.5905 - accuracy: 0.4297 - val_loss: 1.2218 - val_accuracy: 0.5855
Epoch 4/20
75/75 [==============================] - 14s 192ms/step - loss: 1.4620 - accuracy: 0.4754 - val_loss: 1.0449 - val_accuracy: 0.6440
Epoch 5/20
75/75 [==============================] - 14s 191ms/step - loss: 1.3460 - accuracy: 0.5037 - val_loss: 1.0674 - val_accuracy: 0.6557
Epoch 6/20
75/75 [==============================] - 14s 193ms/step - loss: 1.2815 - accuracy: 0.5523 - val_loss: 1.0745 - val_accuracy: 0.6343
Epoch 7/20
75/75 [==============================] - 14s 192ms/step - loss: 1.2304 - accuracy: 0.5639 - val_loss: 1.0559 - val_accuracy: 0.64

In [54]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 22s 176ms/step - loss: 1.8411 - accuracy: 0.4044 - val_loss: 0.6860 - val_accuracy: 0.7964
Epoch 2/20
113/113 [==============================] - 19s 171ms/step - loss: 1.4190 - accuracy: 0.4799 - val_loss: 0.7512 - val_accuracy: 0.7893
Epoch 3/20
113/113 [==============================] - 19s 170ms/step - loss: 1.3426 - accuracy: 0.5040 - val_loss: 0.6850 - val_accuracy: 0.7751
Epoch 4/20
113/113 [==============================] - 19s 169ms/step - loss: 1.3064 - accuracy: 0.5224 - val_loss: 0.7827 - val_accuracy: 0.7498
Epoch 5/20
113/113 [==============================] - 19s 169ms/step - loss: 1.2203 - accuracy: 0.5543 - val_loss: 0.6860 - val_accuracy: 0.7766
Epoch 6/20
113/113 [==============================] - 19s 171ms/step - loss: 1.1895 - accuracy: 0.5632 - val_loss: 0.6979 - val_accuracy: 0.7754
Epoch 7/20
113/113 [==============================] - 19s 170ms/step - loss: 1.0999 - accuracy: 0.6035 - val_loss: 0.7370 - val_

In [55]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 26s 160ms/step - loss: 1.5227 - accuracy: 0.4850 - val_loss: 0.6708 - val_accuracy: 0.7984
Epoch 2/20
150/150 [==============================] - 24s 157ms/step - loss: 1.2706 - accuracy: 0.5441 - val_loss: 0.5854 - val_accuracy: 0.8173
Epoch 3/20
150/150 [==============================] - 23s 157ms/step - loss: 1.1988 - accuracy: 0.5696 - val_loss: 0.6649 - val_accuracy: 0.7932
Epoch 4/20
150/150 [==============================] - 23s 157ms/step - loss: 1.1318 - accuracy: 0.5825 - val_loss: 0.6676 - val_accuracy: 0.7864
Epoch 5/20
150/150 [==============================] - 24s 157ms/step - loss: 1.0656 - accuracy: 0.6209 - val_loss: 0.6318 - val_accuracy: 0.7969
Epoch 6/20
150/150 [==============================] - 24s 157ms/step - loss: 1.0029 - accuracy: 0.6384 - val_loss: 0.6520 - val_accuracy: 0.7744
Epoch 7/20
150/150 [==============================] - 24s 158ms/step - loss: 0.9924 - accuracy: 0.6371 - val_loss: 0.6159 - val_

In [56]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 31s 153ms/step - loss: 1.3787 - accuracy: 0.5381 - val_loss: 0.6210 - val_accuracy: 0.8062
Epoch 2/20
188/188 [==============================] - 29s 154ms/step - loss: 1.1701 - accuracy: 0.5869 - val_loss: 0.6985 - val_accuracy: 0.7933
Epoch 3/20
188/188 [==============================] - 28s 151ms/step - loss: 1.1258 - accuracy: 0.5980 - val_loss: 0.6495 - val_accuracy: 0.7847
Epoch 4/20
188/188 [==============================] - 28s 152ms/step - loss: 1.0617 - accuracy: 0.6223 - val_loss: 0.6897 - val_accuracy: 0.7768
Epoch 5/20
188/188 [==============================] - 28s 151ms/step - loss: 1.0220 - accuracy: 0.6396 - val_loss: 0.6312 - val_accuracy: 0.7998
Epoch 6/20
188/188 [==============================] - 29s 152ms/step - loss: 0.9736 - accuracy: 0.6483 - val_loss: 0.5601 - val_accuracy: 0.8098
Epoch 7/20
188/188 [==============================] - 28s 151ms/step - loss: 0.9214 - accuracy: 0.6634 - val_loss: 0.7162 - val_

In [57]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 36s 152ms/step - loss: 1.2674 - accuracy: 0.5801 - val_loss: 0.6497 - val_accuracy: 0.7985
Epoch 2/20
225/225 [==============================] - 33s 148ms/step - loss: 1.0907 - accuracy: 0.6144 - val_loss: 0.6087 - val_accuracy: 0.8121
Epoch 3/20
225/225 [==============================] - 33s 146ms/step - loss: 1.0133 - accuracy: 0.6327 - val_loss: 0.6874 - val_accuracy: 0.7747
Epoch 4/20
225/225 [==============================] - 33s 147ms/step - loss: 0.9733 - accuracy: 0.6574 - val_loss: 0.6259 - val_accuracy: 0.8020
Epoch 5/20
225/225 [==============================] - 33s 146ms/step - loss: 0.9478 - accuracy: 0.6560 - val_loss: 0.7417 - val_accuracy: 0.7760
Epoch 6/20
225/225 [==============================] - 33s 146ms/step - loss: 0.9007 - accuracy: 0.6800 - val_loss: 0.6802 - val_accuracy: 0.7818
Epoch 7/20
225/225 [==============================] - 33s 146ms/step - loss: 0.9040 - accuracy: 0.6832 - val_loss: 0.5795 - val_

In [58]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 40s 147ms/step - loss: 1.1943 - accuracy: 0.5962 - val_loss: 0.7088 - val_accuracy: 0.7792
Epoch 2/20
263/263 [==============================] - 38s 145ms/step - loss: 1.0273 - accuracy: 0.6460 - val_loss: 0.5812 - val_accuracy: 0.8197
Epoch 3/20
263/263 [==============================] - 38s 145ms/step - loss: 0.9909 - accuracy: 0.6465 - val_loss: 0.5969 - val_accuracy: 0.8104
Epoch 4/20
263/263 [==============================] - 38s 145ms/step - loss: 0.9293 - accuracy: 0.6765 - val_loss: 0.6472 - val_accuracy: 0.7803
Epoch 5/20
263/263 [==============================] - 38s 144ms/step - loss: 0.9230 - accuracy: 0.6719 - val_loss: 0.5356 - val_accuracy: 0.8199
Epoch 6/20
263/263 [==============================] - 38s 146ms/step - loss: 0.8997 - accuracy: 0.6846 - val_loss: 0.8118 - val_accuracy: 0.7359
Epoch 7/20
263/263 [==============================] - 38s 146ms/step - loss: 0.8721 - accuracy: 0.6896 - val_loss: 0.6283 - val_

In [59]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 45s 142ms/step - loss: 1.1600 - accuracy: 0.6229 - val_loss: 0.5417 - val_accuracy: 0.8262
Epoch 2/20
300/300 [==============================] - 43s 142ms/step - loss: 0.9758 - accuracy: 0.6571 - val_loss: 0.7189 - val_accuracy: 0.8000
Epoch 3/20
300/300 [==============================] - 42s 141ms/step - loss: 0.9449 - accuracy: 0.6714 - val_loss: 0.6152 - val_accuracy: 0.7938
Epoch 4/20
300/300 [==============================] - 42s 141ms/step - loss: 0.9041 - accuracy: 0.6827 - val_loss: 0.5533 - val_accuracy: 0.8229
Epoch 5/20
300/300 [==============================] - 42s 140ms/step - loss: 0.8839 - accuracy: 0.6889 - val_loss: 0.6401 - val_accuracy: 0.7967
Epoch 6/20
300/300 [==============================] - 42s 141ms/step - loss: 0.8541 - accuracy: 0.6963 - val_loss: 0.6289 - val_accuracy: 0.8020
Epoch 7/20
300/300 [==============================] - 42s 141ms/step - loss: 0.8439 - accuracy: 0.7029 - val_loss: 0.7045 - val_

In [60]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 49s 140ms/step - loss: 1.0691 - accuracy: 0.6394 - val_loss: 0.7237 - val_accuracy: 0.7832
Epoch 2/20
338/338 [==============================] - 47s 139ms/step - loss: 0.9726 - accuracy: 0.6548 - val_loss: 0.5489 - val_accuracy: 0.8236
Epoch 3/20
338/338 [==============================] - 47s 139ms/step - loss: 0.9227 - accuracy: 0.6725 - val_loss: 0.6214 - val_accuracy: 0.8124
Epoch 4/20
338/338 [==============================] - 47s 139ms/step - loss: 0.8759 - accuracy: 0.6959 - val_loss: 0.7072 - val_accuracy: 0.7753
Epoch 5/20
338/338 [==============================] - 47s 140ms/step - loss: 0.8754 - accuracy: 0.6935 - val_loss: 0.6176 - val_accuracy: 0.8069
Epoch 6/20
338/338 [==============================] - 47s 139ms/step - loss: 0.8441 - accuracy: 0.7029 - val_loss: 0.7787 - val_accuracy: 0.7591
Epoch 7/20
338/338 [==============================] - 47s 140ms/step - loss: 0.8368 - accuracy: 0.7065 - val_loss: 0.5462 - val_

In [61]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 54s 139ms/step - loss: 1.0632 - accuracy: 0.6471 - val_loss: 0.6094 - val_accuracy: 0.8105
Epoch 2/20
375/375 [==============================] - 53s 141ms/step - loss: 0.9360 - accuracy: 0.6798 - val_loss: 0.5880 - val_accuracy: 0.8286
Epoch 3/20
375/375 [==============================] - 52s 138ms/step - loss: 0.8983 - accuracy: 0.6836 - val_loss: 0.5259 - val_accuracy: 0.8198
Epoch 4/20
375/375 [==============================] - 51s 137ms/step - loss: 0.8641 - accuracy: 0.7005 - val_loss: 0.6407 - val_accuracy: 0.8056
Epoch 5/20
375/375 [==============================] - 51s 137ms/step - loss: 0.8523 - accuracy: 0.7019 - val_loss: 0.6257 - val_accuracy: 0.7989
Epoch 6/20
375/375 [==============================] - 51s 137ms/step - loss: 0.8405 - accuracy: 0.7089 - val_loss: 0.5762 - val_accuracy: 0.8104
Epoch 7/20
375/375 [==============================] - 52s 138ms/step - loss: 0.8066 - accuracy: 0.7192 - val_loss: 0.5929 - val_

In [62]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 59s 138ms/step - loss: 1.0680 - accuracy: 0.6455 - val_loss: 0.5839 - val_accuracy: 0.8049
Epoch 2/20
413/413 [==============================] - 57s 138ms/step - loss: 0.9158 - accuracy: 0.6841 - val_loss: 0.5885 - val_accuracy: 0.8142
Epoch 3/20
413/413 [==============================] - 57s 138ms/step - loss: 0.8983 - accuracy: 0.6883 - val_loss: 0.6969 - val_accuracy: 0.7928
Epoch 4/20
413/413 [==============================] - 57s 139ms/step - loss: 0.8489 - accuracy: 0.7042 - val_loss: 0.7545 - val_accuracy: 0.7808
Epoch 5/20
413/413 [==============================] - 57s 138ms/step - loss: 0.8385 - accuracy: 0.7097 - val_loss: 0.8920 - val_accuracy: 0.7676
Epoch 6/20
413/413 [==============================] - 57s 138ms/step - loss: 0.8075 - accuracy: 0.7206 - val_loss: 0.6687 - val_accuracy: 0.7866
Epoch 7/20
413/413 [==============================] - 57s 138ms/step - loss: 0.7968 - accuracy: 0.7242 - val_loss: 0.6442 - val

In [63]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 64s 137ms/step - loss: 0.9935 - accuracy: 0.6640 - val_loss: 0.7135 - val_accuracy: 0.8016
Epoch 2/20
450/450 [==============================] - 61s 135ms/step - loss: 0.8962 - accuracy: 0.6913 - val_loss: 0.6586 - val_accuracy: 0.8080
Epoch 3/20
450/450 [==============================] - 61s 135ms/step - loss: 0.8782 - accuracy: 0.6942 - val_loss: 0.6460 - val_accuracy: 0.8059
Epoch 4/20
450/450 [==============================] - 61s 136ms/step - loss: 0.8518 - accuracy: 0.7104 - val_loss: 0.5794 - val_accuracy: 0.8232
Epoch 5/20
450/450 [==============================] - 61s 136ms/step - loss: 0.8226 - accuracy: 0.7147 - val_loss: 0.5791 - val_accuracy: 0.8148
Epoch 6/20
450/450 [==============================] - 61s 135ms/step - loss: 0.7859 - accuracy: 0.7289 - val_loss: 0.6588 - val_accuracy: 0.8027
Epoch 7/20
450/450 [==============================] - 64s 143ms/step - loss: 0.7931 - accuracy: 0.7280 - val_loss: 0.5672 - val

In [64]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 70s 138ms/step - loss: 0.9879 - accuracy: 0.6777 - val_loss: 0.5922 - val_accuracy: 0.8168
Epoch 2/20
488/488 [==============================] - 67s 137ms/step - loss: 0.8887 - accuracy: 0.6933 - val_loss: 0.7078 - val_accuracy: 0.7769
Epoch 3/20
488/488 [==============================] - 67s 137ms/step - loss: 0.8421 - accuracy: 0.7128 - val_loss: 0.6412 - val_accuracy: 0.8013
Epoch 4/20
488/488 [==============================] - 68s 139ms/step - loss: 0.8344 - accuracy: 0.7122 - val_loss: 0.6794 - val_accuracy: 0.7952
Epoch 5/20
488/488 [==============================] - 67s 138ms/step - loss: 0.8062 - accuracy: 0.7246 - val_loss: 0.6963 - val_accuracy: 0.7783
Epoch 6/20
488/488 [==============================] - 67s 138ms/step - loss: 0.7990 - accuracy: 0.7217 - val_loss: 0.6301 - val_accuracy: 0.7889
Epoch 7/20
488/488 [==============================] - 67s 137ms/step - loss: 0.7833 - accuracy: 0.7303 - val_loss: 0.6745 - val

In [65]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 73s 135ms/step - loss: 0.9787 - accuracy: 0.6742 - val_loss: 0.6774 - val_accuracy: 0.8226
Epoch 2/20
525/525 [==============================] - 71s 135ms/step - loss: 0.8743 - accuracy: 0.6992 - val_loss: 0.5653 - val_accuracy: 0.8299
Epoch 3/20
525/525 [==============================] - 71s 135ms/step - loss: 0.8291 - accuracy: 0.7147 - val_loss: 0.5947 - val_accuracy: 0.8115
Epoch 4/20
525/525 [==============================] - 71s 136ms/step - loss: 0.8261 - accuracy: 0.7161 - val_loss: 0.5954 - val_accuracy: 0.8073
Epoch 5/20
525/525 [==============================] - 71s 136ms/step - loss: 0.8100 - accuracy: 0.7164 - val_loss: 0.5886 - val_accuracy: 0.8153
Epoch 6/20
525/525 [==============================] - 72s 137ms/step - loss: 0.7865 - accuracy: 0.7292 - val_loss: 0.5710 - val_accuracy: 0.8124
Epoch 7/20
525/525 [==============================] - 72s 136ms/step - loss: 0.7758 - accuracy: 0.7330 - val_loss: 0.5695 - val

In [66]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 79s 138ms/step - loss: 0.9487 - accuracy: 0.6870 - val_loss: 0.6244 - val_accuracy: 0.7967
Epoch 2/20
563/563 [==============================] - 75s 134ms/step - loss: 0.8673 - accuracy: 0.7043 - val_loss: 0.5951 - val_accuracy: 0.8036
Epoch 3/20
563/563 [==============================] - 76s 135ms/step - loss: 0.8361 - accuracy: 0.7111 - val_loss: 0.6188 - val_accuracy: 0.8075
Epoch 4/20
563/563 [==============================] - 76s 135ms/step - loss: 0.8040 - accuracy: 0.7194 - val_loss: 0.5504 - val_accuracy: 0.8218
Epoch 5/20
563/563 [==============================] - 76s 135ms/step - loss: 0.7908 - accuracy: 0.7283 - val_loss: 0.6555 - val_accuracy: 0.7916
Epoch 6/20
563/563 [==============================] - 77s 136ms/step - loss: 0.7809 - accuracy: 0.7326 - val_loss: 0.6114 - val_accuracy: 0.8077
Epoch 7/20
563/563 [==============================] - 76s 136ms/step - loss: 0.7656 - accuracy: 0.7348 - val_loss: 0.6480 - val

In [67]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 82s 133ms/step - loss: 0.9290 - accuracy: 0.6905 - val_loss: 0.5630 - val_accuracy: 0.8267
Epoch 2/20
600/600 [==============================] - 80s 134ms/step - loss: 0.8522 - accuracy: 0.7088 - val_loss: 0.5565 - val_accuracy: 0.8286
Epoch 3/20
600/600 [==============================] - 80s 134ms/step - loss: 0.8267 - accuracy: 0.7113 - val_loss: 0.6014 - val_accuracy: 0.8054
Epoch 4/20
600/600 [==============================] - 81s 135ms/step - loss: 0.8063 - accuracy: 0.7248 - val_loss: 0.7423 - val_accuracy: 0.7640
Epoch 5/20
600/600 [==============================] - 80s 134ms/step - loss: 0.7942 - accuracy: 0.7271 - val_loss: 0.6008 - val_accuracy: 0.8129
Epoch 6/20
600/600 [==============================] - 80s 134ms/step - loss: 0.7803 - accuracy: 0.7330 - val_loss: 0.6926 - val_accuracy: 0.7797
Epoch 7/20
600/600 [==============================] - 81s 134ms/step - loss: 0.7676 - accuracy: 0.7343 - val_loss: 0.5839 - val

In [68]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 93s 142ms/step - loss: 0.9091 - accuracy: 0.6951 - val_loss: 0.6012 - val_accuracy: 0.8150
Epoch 2/20
638/638 [==============================] - 91s 143ms/step - loss: 0.8398 - accuracy: 0.7143 - val_loss: 0.5419 - val_accuracy: 0.8228
Epoch 3/20
638/638 [==============================] - 92s 144ms/step - loss: 0.8242 - accuracy: 0.7191 - val_loss: 0.7231 - val_accuracy: 0.7929
Epoch 4/20
638/638 [==============================] - 91s 143ms/step - loss: 0.8103 - accuracy: 0.7210 - val_loss: 0.6132 - val_accuracy: 0.8108
Epoch 5/20
638/638 [==============================] - 92s 144ms/step - loss: 0.7699 - accuracy: 0.7313 - val_loss: 0.6559 - val_accuracy: 0.7883
Epoch 6/20
638/638 [==============================] - 92s 144ms/step - loss: 0.7748 - accuracy: 0.7337 - val_loss: 0.6479 - val_accuracy: 0.7971
Epoch 7/20
638/638 [==============================] - 91s 143ms/step - loss: 0.7788 - accuracy: 0.7305 - val_loss: 0.6164 - val

In [69]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 98s 143ms/step - loss: 0.9213 - accuracy: 0.6997 - val_loss: 0.5343 - val_accuracy: 0.8290
Epoch 2/20
675/675 [==============================] - 95s 141ms/step - loss: 0.8198 - accuracy: 0.7192 - val_loss: 0.6730 - val_accuracy: 0.8138
Epoch 3/20
675/675 [==============================] - 95s 141ms/step - loss: 0.8054 - accuracy: 0.7225 - val_loss: 0.6839 - val_accuracy: 0.8026
Epoch 4/20
675/675 [==============================] - 95s 141ms/step - loss: 0.7941 - accuracy: 0.7258 - val_loss: 0.6719 - val_accuracy: 0.8024
Epoch 5/20
675/675 [==============================] - 96s 142ms/step - loss: 0.7809 - accuracy: 0.7340 - val_loss: 0.7000 - val_accuracy: 0.8069
Epoch 6/20
675/675 [==============================] - 95s 141ms/step - loss: 0.7645 - accuracy: 0.7403 - val_loss: 0.6039 - val_accuracy: 0.8030
Epoch 7/20
675/675 [==============================] - 96s 142ms/step - loss: 0.7698 - accuracy: 0.7373 - val_loss: 0.6321 - val

In [70]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 121s 166ms/step - loss: 0.8967 - accuracy: 0.7021 - val_loss: 0.7131 - val_accuracy: 0.7911
Epoch 2/20
713/713 [==============================] - 116s 163ms/step - loss: 0.8105 - accuracy: 0.7202 - val_loss: 0.6229 - val_accuracy: 0.8007
Epoch 3/20
713/713 [==============================] - 109s 153ms/step - loss: 0.8101 - accuracy: 0.7244 - val_loss: 0.6465 - val_accuracy: 0.8083
Epoch 4/20
713/713 [==============================] - 102s 143ms/step - loss: 0.7967 - accuracy: 0.7236 - val_loss: 0.5975 - val_accuracy: 0.8123
Epoch 5/20
713/713 [==============================] - 99s 139ms/step - loss: 0.7756 - accuracy: 0.7341 - val_loss: 0.7505 - val_accuracy: 0.7942
Epoch 6/20
713/713 [==============================] - 119s 168ms/step - loss: 0.7667 - accuracy: 0.7341 - val_loss: 0.6521 - val_accuracy: 0.7897
Epoch 7/20
713/713 [==============================] - 105s 148ms/step - loss: 0.7640 - accuracy: 0.7397 - val_loss: 0.6455

In [71]:
time.sleep(400)

In [72]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 74s 96ms/step - loss: 0.8976 - accuracy: 0.7018 - val_loss: 0.5485 - val_accuracy: 0.8240
Epoch 2/20
750/750 [==============================] - 99s 131ms/step - loss: 0.8261 - accuracy: 0.7154 - val_loss: 0.6331 - val_accuracy: 0.8115
Epoch 3/20
750/750 [==============================] - 95s 127ms/step - loss: 0.8141 - accuracy: 0.7233 - val_loss: 0.6447 - val_accuracy: 0.8063
Epoch 4/20
750/750 [==============================] - 98s 131ms/step - loss: 0.7880 - accuracy: 0.7301 - val_loss: 0.6217 - val_accuracy: 0.8095
Epoch 5/20
750/750 [==============================] - 96s 128ms/step - loss: 0.7699 - accuracy: 0.7352 - val_loss: 0.6319 - val_accuracy: 0.8077
Epoch 6/20
750/750 [==============================] - 100s 134ms/step - loss: 0.7720 - accuracy: 0.7387 - val_loss: 0.5657 - val_accuracy: 0.8086
Epoch 7/20
750/750 [==============================] - 98s 130ms/step - loss: 0.7642 - accuracy: 0.7413 - val_loss: 0.6627 - val

In [ ]:
time.sleep(400)

In [73]:
new_model_dsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_dsa_e1"

i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets writ

In [ ]:
loading = False

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [74]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [75]:
gc.collect()

508542

## Uncertainty

In [19]:
dg_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)


In [20]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_train_and_adversary)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_train_and_adversary)

In [21]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [22]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dg.append(model)

C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [80]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 15s 344ms/step - loss: 1.8765 - accuracy: 0.3410 - val_loss: 1.0011 - val_accuracy: 0.7501
Epoch 2/20
38/38 [==============================] - 12s 318ms/step - loss: 1.6143 - accuracy: 0.4024 - val_loss: 0.7332 - val_accuracy: 0.7711
Epoch 3/20
38/38 [==============================] - 12s 319ms/step - loss: 1.5191 - accuracy: 0.4125 - val_loss: 0.7668 - val_accuracy: 0.7811
Epoch 4/20
38/38 [==============================] - 12s 314ms/step - loss: 1.4696 - accuracy: 0.4511 - val_loss: 0.9017 - val_accuracy: 0.7218
Epoch 5/20
38/38 [==============================] - 12s 320ms/step - loss: 1.3117 - accuracy: 0.5136 - val_loss: 0.6890 - val_accuracy: 0.8188
Epoch 6/20
38/38 [==============================] - 12s 315ms/step - loss: 1.2971 - accuracy: 0.5163 - val_loss: 0.6518 - val_accuracy: 0.8133
Epoch 7/20
38/38 [==============================] - 12s 317ms/step - loss: 1.1790 - accuracy: 0.5612 - val_loss: 0.6744 - val_accuracy: 0.80

In [81]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 19s 230ms/step - loss: 1.7615 - accuracy: 0.3708 - val_loss: 0.7402 - val_accuracy: 0.7722
Epoch 2/20
75/75 [==============================] - 16s 218ms/step - loss: 1.5289 - accuracy: 0.4434 - val_loss: 0.7801 - val_accuracy: 0.8140
Epoch 3/20
75/75 [==============================] - 16s 217ms/step - loss: 1.4692 - accuracy: 0.4570 - val_loss: 0.6630 - val_accuracy: 0.8059
Epoch 4/20
75/75 [==============================] - 16s 219ms/step - loss: 1.3903 - accuracy: 0.4889 - val_loss: 0.8629 - val_accuracy: 0.7466
Epoch 5/20
75/75 [==============================] - 16s 220ms/step - loss: 1.3285 - accuracy: 0.5151 - val_loss: 0.7485 - val_accuracy: 0.7661
Epoch 6/20
75/75 [==============================] - 16s 219ms/step - loss: 1.2926 - accuracy: 0.5220 - val_loss: 0.7469 - val_accuracy: 0.7757
Epoch 7/20
75/75 [==============================] - 16s 221ms/step - loss: 1.1989 - accuracy: 0.5647 - val_loss: 0.6869 - val_accuracy: 0.78

In [82]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 23s 182ms/step - loss: 1.6562 - accuracy: 0.4008 - val_loss: 0.8166 - val_accuracy: 0.7646
Epoch 2/20
113/113 [==============================] - 20s 178ms/step - loss: 1.5115 - accuracy: 0.4459 - val_loss: 0.9534 - val_accuracy: 0.7464
Epoch 3/20
113/113 [==============================] - 20s 179ms/step - loss: 1.4327 - accuracy: 0.4754 - val_loss: 0.7348 - val_accuracy: 0.8083
Epoch 4/20
113/113 [==============================] - 20s 177ms/step - loss: 1.3929 - accuracy: 0.4957 - val_loss: 0.6409 - val_accuracy: 0.8141
Epoch 5/20
113/113 [==============================] - 20s 178ms/step - loss: 1.3036 - accuracy: 0.5235 - val_loss: 0.6970 - val_accuracy: 0.8117
Epoch 6/20
113/113 [==============================] - 20s 178ms/step - loss: 1.2703 - accuracy: 0.5390 - val_loss: 0.6529 - val_accuracy: 0.7972
Epoch 7/20
113/113 [==============================] - 20s 178ms/step - loss: 1.2176 - accuracy: 0.5537 - val_loss: 0.6933 - val_

In [83]:

print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 26s 162ms/step - loss: 1.6758 - accuracy: 0.4006 - val_loss: 0.6961 - val_accuracy: 0.8274
Epoch 2/20
150/150 [==============================] - 24s 160ms/step - loss: 1.4740 - accuracy: 0.4591 - val_loss: 0.7551 - val_accuracy: 0.7867
Epoch 3/20
150/150 [==============================] - 26s 174ms/step - loss: 1.3927 - accuracy: 0.4961 - val_loss: 0.6340 - val_accuracy: 0.8180
Epoch 4/20
150/150 [==============================] - 23s 156ms/step - loss: 1.3631 - accuracy: 0.4987 - val_loss: 0.6778 - val_accuracy: 0.8047
Epoch 5/20
150/150 [==============================] - 24s 159ms/step - loss: 1.2922 - accuracy: 0.5302 - val_loss: 0.7305 - val_accuracy: 0.7669
Epoch 6/20
150/150 [==============================] - 24s 161ms/step - loss: 1.2458 - accuracy: 0.5463 - val_loss: 0.6234 - val_accuracy: 0.8157
Epoch 7/20
150/150 [==============================] - 24s 162ms/step - loss: 1.1993 - accuracy: 0.5609 - val_loss: 0.6788 - val_

In [85]:
n

4

In [21]:
n=4
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 73s 303ms/step - loss: 1.6264 - accuracy: 0.4136 - val_loss: 0.7979 - val_accuracy: 0.8093
Epoch 2/20
188/188 [==============================] - 55s 291ms/step - loss: 1.4269 - accuracy: 0.4823 - val_loss: 0.7358 - val_accuracy: 0.7882
Epoch 3/20
188/188 [==============================] - 53s 282ms/step - loss: 1.3764 - accuracy: 0.4986 - val_loss: 0.8460 - val_accuracy: 0.8010
Epoch 4/20
188/188 [==============================] - 52s 277ms/step - loss: 1.3227 - accuracy: 0.5120 - val_loss: 0.7277 - val_accuracy: 0.8033
Epoch 5/20
188/188 [==============================] - 51s 273ms/step - loss: 1.2747 - accuracy: 0.5303 - val_loss: 0.6834 - val_accuracy: 0.7956
Epoch 6/20
188/188 [==============================] - 51s 274ms/step - loss: 1.2450 - accuracy: 0.5402 - val_loss: 0.7528 - val_accuracy: 0.7718
Epoch 7/20
188/188 [==============================] - 52s 277ms/step - loss: 1.1839 - accuracy: 0.5660 - val_loss: 0.5828 - val_

In [22]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 70s 298ms/step - loss: 1.5676 - accuracy: 0.4336 - val_loss: 0.7388 - val_accuracy: 0.8178
Epoch 2/20
225/225 [==============================] - 66s 294ms/step - loss: 1.3939 - accuracy: 0.4856 - val_loss: 0.7410 - val_accuracy: 0.7727
Epoch 3/20
225/225 [==============================] - 66s 294ms/step - loss: 1.3403 - accuracy: 0.5046 - val_loss: 0.7114 - val_accuracy: 0.8179
Epoch 4/20
225/225 [==============================] - 66s 294ms/step - loss: 1.2957 - accuracy: 0.5250 - val_loss: 0.6856 - val_accuracy: 0.7875
Epoch 5/20
225/225 [==============================] - 66s 295ms/step - loss: 1.2370 - accuracy: 0.5463 - val_loss: 0.6775 - val_accuracy: 0.8053
Epoch 6/20
225/225 [==============================] - 66s 293ms/step - loss: 1.2059 - accuracy: 0.5618 - val_loss: 0.6616 - val_accuracy: 0.8105
Epoch 7/20
225/225 [==============================] - 66s 295ms/step - loss: 1.1900 - accuracy: 0.5761 - val_loss: 0.6465 - val_

In [23]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 80s 293ms/step - loss: 1.5048 - accuracy: 0.4639 - val_loss: 0.7057 - val_accuracy: 0.7984
Epoch 2/20
263/263 [==============================] - 76s 290ms/step - loss: 1.3542 - accuracy: 0.5083 - val_loss: 0.8515 - val_accuracy: 0.7963
Epoch 3/20
263/263 [==============================] - 77s 293ms/step - loss: 1.2992 - accuracy: 0.5233 - val_loss: 0.8132 - val_accuracy: 0.7872
Epoch 4/20
263/263 [==============================] - 76s 289ms/step - loss: 1.2599 - accuracy: 0.5452 - val_loss: 0.6739 - val_accuracy: 0.8102
Epoch 5/20
263/263 [==============================] - 78s 296ms/step - loss: 1.2206 - accuracy: 0.5547 - val_loss: 0.7015 - val_accuracy: 0.7839
Epoch 6/20
263/263 [==============================] - 77s 291ms/step - loss: 1.1916 - accuracy: 0.5623 - val_loss: 0.6658 - val_accuracy: 0.7975
Epoch 7/20
263/263 [==============================] - 76s 291ms/step - loss: 1.1486 - accuracy: 0.5806 - val_loss: 0.6619 - val_

In [24]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 94s 303ms/step - loss: 1.4819 - accuracy: 0.4752 - val_loss: 0.7419 - val_accuracy: 0.7980
Epoch 2/20
300/300 [==============================] - 90s 301ms/step - loss: 1.3282 - accuracy: 0.5190 - val_loss: 0.6502 - val_accuracy: 0.8150
Epoch 3/20
300/300 [==============================] - 89s 295ms/step - loss: 1.2645 - accuracy: 0.5414 - val_loss: 0.6973 - val_accuracy: 0.7912
Epoch 4/20
300/300 [==============================] - 85s 284ms/step - loss: 1.2417 - accuracy: 0.5502 - val_loss: 0.7492 - val_accuracy: 0.7982
Epoch 5/20
300/300 [==============================] - 84s 282ms/step - loss: 1.1955 - accuracy: 0.5622 - val_loss: 0.6409 - val_accuracy: 0.8123
Epoch 6/20
300/300 [==============================] - 86s 286ms/step - loss: 1.1566 - accuracy: 0.5830 - val_loss: 0.6446 - val_accuracy: 0.8060
Epoch 7/20
300/300 [==============================] - 86s 286ms/step - loss: 1.1347 - accuracy: 0.5899 - val_loss: 0.7270 - val_

In [25]:
n=8
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 98s 281ms/step - loss: 1.4297 - accuracy: 0.4991 - val_loss: 0.6049 - val_accuracy: 0.8133
Epoch 2/20
338/338 [==============================] - 94s 278ms/step - loss: 1.2886 - accuracy: 0.5347 - val_loss: 0.6415 - val_accuracy: 0.8143
Epoch 3/20
338/338 [==============================] - 96s 283ms/step - loss: 1.2326 - accuracy: 0.5554 - val_loss: 0.6172 - val_accuracy: 0.8088
Epoch 4/20
338/338 [==============================] - 95s 281ms/step - loss: 1.1945 - accuracy: 0.5714 - val_loss: 0.6588 - val_accuracy: 0.8155
Epoch 5/20
338/338 [==============================] - 95s 281ms/step - loss: 1.1710 - accuracy: 0.5780 - val_loss: 0.8729 - val_accuracy: 0.7591
Epoch 6/20
338/338 [==============================] - 94s 279ms/step - loss: 1.1301 - accuracy: 0.5889 - val_loss: 0.6312 - val_accuracy: 0.8093
Epoch 7/20
338/338 [==============================] - 95s 281ms/step - loss: 1.1045 - accuracy: 0.5998 - val_loss: 0.6222 - val_

In [26]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 106s 275ms/step - loss: 1.3917 - accuracy: 0.5203 - val_loss: 0.6568 - val_accuracy: 0.8257
Epoch 2/20
375/375 [==============================] - 104s 277ms/step - loss: 1.2404 - accuracy: 0.5557 - val_loss: 0.6147 - val_accuracy: 0.8092
Epoch 3/20
375/375 [==============================] - 103s 275ms/step - loss: 1.1970 - accuracy: 0.5786 - val_loss: 0.7024 - val_accuracy: 0.7939
Epoch 4/20
375/375 [==============================] - 102s 272ms/step - loss: 1.1644 - accuracy: 0.5877 - val_loss: 0.6945 - val_accuracy: 0.8023
Epoch 5/20
375/375 [==============================] - 102s 273ms/step - loss: 1.1286 - accuracy: 0.5961 - val_loss: 0.6903 - val_accuracy: 0.7950
Epoch 6/20
375/375 [==============================] - 103s 274ms/step - loss: 1.1102 - accuracy: 0.6009 - val_loss: 0.5786 - val_accuracy: 0.8181
Epoch 7/20
375/375 [==============================] - 103s 275ms/step - loss: 1.0877 - accuracy: 0.6141 - val_loss: 0.5938

In [27]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 111s 262ms/step - loss: 1.3357 - accuracy: 0.5385 - val_loss: 0.6753 - val_accuracy: 0.8162
Epoch 2/20
413/413 [==============================] - 108s 262ms/step - loss: 1.1895 - accuracy: 0.5753 - val_loss: 0.6383 - val_accuracy: 0.7900
Epoch 3/20
413/413 [==============================] - 108s 261ms/step - loss: 1.1492 - accuracy: 0.5946 - val_loss: 0.6123 - val_accuracy: 0.8182
Epoch 4/20
413/413 [==============================] - 108s 262ms/step - loss: 1.1050 - accuracy: 0.6061 - val_loss: 0.5970 - val_accuracy: 0.8073
Epoch 5/20
413/413 [==============================] - 107s 260ms/step - loss: 1.0920 - accuracy: 0.6111 - val_loss: 0.5897 - val_accuracy: 0.8068
Epoch 6/20
413/413 [==============================] - 108s 261ms/step - loss: 1.0671 - accuracy: 0.6171 - val_loss: 0.7024 - val_accuracy: 0.7977
Epoch 7/20
413/413 [==============================] - 108s 261ms/step - loss: 1.0396 - accuracy: 0.6301 - val_loss: 0.699

In [28]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 121s 262ms/step - loss: 1.2847 - accuracy: 0.5653 - val_loss: 0.6923 - val_accuracy: 0.8158
Epoch 2/20
450/450 [==============================] - 117s 259ms/step - loss: 1.1577 - accuracy: 0.5955 - val_loss: 0.6760 - val_accuracy: 0.8267
Epoch 3/20
450/450 [==============================] - 117s 261ms/step - loss: 1.1066 - accuracy: 0.6096 - val_loss: 0.5747 - val_accuracy: 0.8208
Epoch 4/20
450/450 [==============================] - 117s 260ms/step - loss: 1.0829 - accuracy: 0.6204 - val_loss: 0.6753 - val_accuracy: 0.8056
Epoch 5/20
450/450 [==============================] - 117s 260ms/step - loss: 1.0493 - accuracy: 0.6307 - val_loss: 0.6753 - val_accuracy: 0.8089
Epoch 6/20
450/450 [==============================] - 117s 261ms/step - loss: 1.0278 - accuracy: 0.6321 - val_loss: 0.5968 - val_accuracy: 0.8179
Epoch 7/20
450/450 [==============================] - 116s 259ms/step - loss: 1.0026 - accuracy: 0.6460 - val_loss: 0.554

In [29]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 136s 272ms/step - loss: 1.2202 - accuracy: 0.5894 - val_loss: 0.5807 - val_accuracy: 0.8196
Epoch 2/20
488/488 [==============================] - 134s 276ms/step - loss: 1.1006 - accuracy: 0.6146 - val_loss: 0.6117 - val_accuracy: 0.8199
Epoch 3/20
488/488 [==============================] - 135s 277ms/step - loss: 1.0631 - accuracy: 0.6290 - val_loss: 0.6225 - val_accuracy: 0.8190
Epoch 4/20
488/488 [==============================] - 134s 275ms/step - loss: 1.0393 - accuracy: 0.6339 - val_loss: 0.8102 - val_accuracy: 0.7703
Epoch 5/20
488/488 [==============================] - 135s 277ms/step - loss: 1.0165 - accuracy: 0.6405 - val_loss: 0.5762 - val_accuracy: 0.8198
Epoch 6/20
488/488 [==============================] - 135s 276ms/step - loss: 1.0093 - accuracy: 0.6452 - val_loss: 0.7309 - val_accuracy: 0.7701
Epoch 7/20
488/488 [==============================] - 130s 267ms/step - loss: 0.9859 - accuracy: 0.6570 - val_loss: 0.628

In [30]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 143s 268ms/step - loss: 1.1618 - accuracy: 0.6018 - val_loss: 0.5956 - val_accuracy: 0.8146
Epoch 2/20
525/525 [==============================] - 136s 260ms/step - loss: 1.0553 - accuracy: 0.6307 - val_loss: 0.6163 - val_accuracy: 0.8225
Epoch 3/20
525/525 [==============================] - 139s 264ms/step - loss: 1.0103 - accuracy: 0.6481 - val_loss: 0.5869 - val_accuracy: 0.8266
Epoch 4/20
525/525 [==============================] - 137s 261ms/step - loss: 0.9912 - accuracy: 0.6592 - val_loss: 0.5524 - val_accuracy: 0.8274
Epoch 5/20
525/525 [==============================] - 137s 261ms/step - loss: 0.9711 - accuracy: 0.6612 - val_loss: 0.6369 - val_accuracy: 0.8046
Epoch 6/20
525/525 [==============================] - 137s 261ms/step - loss: 0.9667 - accuracy: 0.6631 - val_loss: 0.6155 - val_accuracy: 0.8090
Epoch 7/20
525/525 [==============================] - 138s 262ms/step - loss: 0.9373 - accuracy: 0.6730 - val_loss: 0.567

In [31]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 152s 266ms/step - loss: 1.1145 - accuracy: 0.6219 - val_loss: 0.6410 - val_accuracy: 0.8170
Epoch 2/20
563/563 [==============================] - 148s 262ms/step - loss: 1.0086 - accuracy: 0.6477 - val_loss: 0.5275 - val_accuracy: 0.8235
Epoch 3/20
563/563 [==============================] - 148s 263ms/step - loss: 0.9795 - accuracy: 0.6580 - val_loss: 0.5508 - val_accuracy: 0.8103
Epoch 4/20
563/563 [==============================] - 147s 262ms/step - loss: 0.9643 - accuracy: 0.6666 - val_loss: 0.6517 - val_accuracy: 0.8117
Epoch 5/20
563/563 [==============================] - 148s 263ms/step - loss: 0.9437 - accuracy: 0.6722 - val_loss: 0.6098 - val_accuracy: 0.8108
Epoch 6/20
563/563 [==============================] - 149s 265ms/step - loss: 0.9376 - accuracy: 0.6784 - val_loss: 0.7036 - val_accuracy: 0.7914
Epoch 7/20
563/563 [==============================] - 149s 264ms/step - loss: 0.9069 - accuracy: 0.6884 - val_loss: 0.680

In [32]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 166s 272ms/step - loss: 1.0790 - accuracy: 0.6426 - val_loss: 0.5481 - val_accuracy: 0.8330
Epoch 2/20
600/600 [==============================] - 162s 269ms/step - loss: 0.9743 - accuracy: 0.6636 - val_loss: 0.5958 - val_accuracy: 0.8134
Epoch 3/20
600/600 [==============================] - 162s 270ms/step - loss: 0.9326 - accuracy: 0.6803 - val_loss: 0.5447 - val_accuracy: 0.8121
Epoch 4/20
600/600 [==============================] - 162s 270ms/step - loss: 0.9198 - accuracy: 0.6805 - val_loss: 0.6120 - val_accuracy: 0.8202
Epoch 5/20
600/600 [==============================] - 162s 271ms/step - loss: 0.9022 - accuracy: 0.6902 - val_loss: 0.7151 - val_accuracy: 0.7918
Epoch 6/20
600/600 [==============================] - 164s 273ms/step - loss: 0.8921 - accuracy: 0.6914 - val_loss: 0.6920 - val_accuracy: 0.8019
Epoch 7/20
600/600 [==============================] - 166s 277ms/step - loss: 0.8818 - accuracy: 0.6973 - val_loss: 0.649

In [33]:
time.sleep(400)

In [34]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 173s 270ms/step - loss: 1.0198 - accuracy: 0.6622 - val_loss: 0.6378 - val_accuracy: 0.8005
Epoch 2/20
638/638 [==============================] - 174s 272ms/step - loss: 0.9331 - accuracy: 0.6834 - val_loss: 0.6269 - val_accuracy: 0.8175
Epoch 3/20
638/638 [==============================] - 175s 274ms/step - loss: 0.9049 - accuracy: 0.6873 - val_loss: 0.6627 - val_accuracy: 0.7845
Epoch 4/20
638/638 [==============================] - 180s 282ms/step - loss: 0.8863 - accuracy: 0.6960 - val_loss: 0.6127 - val_accuracy: 0.8033
Epoch 5/20
638/638 [==============================] - 173s 271ms/step - loss: 0.8753 - accuracy: 0.6969 - val_loss: 0.6104 - val_accuracy: 0.7946
Epoch 6/20
638/638 [==============================] - 171s 268ms/step - loss: 0.8553 - accuracy: 0.7063 - val_loss: 0.6685 - val_accuracy: 0.7845
Epoch 7/20
638/638 [==============================] - 171s 268ms/step - loss: 0.8467 - accuracy: 0.7075 - val_loss: 0.658

In [ ]:
time.sleep(400)

In [35]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 183s 267ms/step - loss: 0.9672 - accuracy: 0.6782 - val_loss: 0.5794 - val_accuracy: 0.8257
Epoch 2/20
675/675 [==============================] - 179s 265ms/step - loss: 0.8903 - accuracy: 0.6934 - val_loss: 0.6285 - val_accuracy: 0.8199
Epoch 3/20
675/675 [==============================] - 179s 266ms/step - loss: 0.8708 - accuracy: 0.6988 - val_loss: 0.5849 - val_accuracy: 0.8134
Epoch 4/20
675/675 [==============================] - 180s 267ms/step - loss: 0.8459 - accuracy: 0.7095 - val_loss: 0.5960 - val_accuracy: 0.8179
Epoch 5/20
675/675 [==============================] - 181s 268ms/step - loss: 0.8338 - accuracy: 0.7156 - val_loss: 0.6426 - val_accuracy: 0.7962
Epoch 6/20
675/675 [==============================] - 182s 269ms/step - loss: 0.8136 - accuracy: 0.7208 - val_loss: 0.6720 - val_accuracy: 0.8045
Epoch 7/20
675/675 [==============================] - 180s 267ms/step - loss: 0.8115 - accuracy: 0.7190 - val_loss: 0.566

In [ ]:
time.sleep(400)

In [23]:
n=18
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 106s 133ms/step - loss: 0.9286 - accuracy: 0.6906 - val_loss: 0.5829 - val_accuracy: 0.8176
Epoch 2/20
713/713 [==============================] - 89s 124ms/step - loss: 0.8529 - accuracy: 0.7065 - val_loss: 0.7471 - val_accuracy: 0.7833
Epoch 3/20
713/713 [==============================] - 96s 134ms/step - loss: 0.8320 - accuracy: 0.7173 - val_loss: 0.5496 - val_accuracy: 0.8285
Epoch 4/20
713/713 [==============================] - 94s 132ms/step - loss: 0.8290 - accuracy: 0.7134 - val_loss: 0.5690 - val_accuracy: 0.8140
Epoch 5/20
713/713 [==============================] - 135s 189ms/step - loss: 0.7956 - accuracy: 0.7246 - val_loss: 0.5914 - val_accuracy: 0.8095
Epoch 6/20
713/713 [==============================] - 173s 242ms/step - loss: 0.7946 - accuracy: 0.7272 - val_loss: 0.5544 - val_accuracy: 0.8137
Epoch 7/20
713/713 [==============================] - 178s 250ms/step - loss: 0.7791 - accuracy: 0.7323 - val_loss: 0.6462 -

In [ ]:
time.sleep(400)

In [24]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 242s 315ms/step - loss: 0.8892 - accuracy: 0.7064 - val_loss: 0.6355 - val_accuracy: 0.8062
Epoch 2/20
750/750 [==============================] - 251s 335ms/step - loss: 0.8244 - accuracy: 0.7190 - val_loss: 0.6371 - val_accuracy: 0.8047
Epoch 3/20
750/750 [==============================] - 230s 306ms/step - loss: 0.8094 - accuracy: 0.7268 - val_loss: 0.6062 - val_accuracy: 0.8086
Epoch 4/20
750/750 [==============================] - 248s 330ms/step - loss: 0.7812 - accuracy: 0.7329 - val_loss: 0.5964 - val_accuracy: 0.8023
Epoch 5/20
750/750 [==============================] - 241s 322ms/step - loss: 0.7824 - accuracy: 0.7339 - val_loss: 0.7111 - val_accuracy: 0.7691
Epoch 6/20
750/750 [==============================] - 248s 330ms/step - loss: 0.7733 - accuracy: 0.7374 - val_loss: 0.6519 - val_accuracy: 0.8002
Epoch 7/20
750/750 [==============================] - 239s 318ms/step - loss: 0.7532 - accuracy: 0.7414 - val_loss: 0.597

In [24]:
new_model_dg_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_dg_e1"


i=18

for model in models_dg[18:19]:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_dg_e1_18\assets
Model has been saved


In [ ]:
loading = False

models_dg = []

if loading:
    for i in range(5):
        model_dg_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_gn_e1_"+str(i)
        print(model_dg_dir)
        model =utils.My_model('gtsrb',True,model_dg_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dg.append(model)
       
    

In [ ]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [ ]:
import gc
gc.collect()

### Softmax 

In [19]:
#softmax values
se_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)


In [20]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_train_and_adversary)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_train_and_adversary)

In [21]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [22]:
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_se.append(model)

0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [23]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 39s 552ms/step - loss: 1.8679 - accuracy: 0.3638 - val_loss: 0.7234 - val_accuracy: 0.8079
Epoch 2/20
38/38 [==============================] - 22s 581ms/step - loss: 1.6148 - accuracy: 0.4213 - val_loss: 0.8316 - val_accuracy: 0.7716
Epoch 3/20
38/38 [==============================] - 23s 622ms/step - loss: 1.5025 - accuracy: 0.4505 - val_loss: 0.8131 - val_accuracy: 0.7599
Epoch 4/20
38/38 [==============================] - 18s 486ms/step - loss: 1.4245 - accuracy: 0.4818 - val_loss: 0.7865 - val_accuracy: 0.8000
Epoch 5/20
38/38 [==============================] - 19s 505ms/step - loss: 1.3680 - accuracy: 0.4898 - val_loss: 0.6419 - val_accuracy: 0.8177
Epoch 6/20
38/38 [==============================] - 18s 487ms/step - loss: 1.2793 - accuracy: 0.5415 - val_loss: 0.7911 - val_accuracy: 0.7540
Epoch 7/20
38/38 [==============================] - 19s 498ms/step - loss: 1.2220 - accuracy: 0.5567 - val_loss: 0.5563 - val_accuracy: 0.82

In [24]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 29s 363ms/step - loss: 1.8344 - accuracy: 0.3724 - val_loss: 0.8842 - val_accuracy: 0.7856
Epoch 2/20
75/75 [==============================] - 27s 355ms/step - loss: 1.5406 - accuracy: 0.4294 - val_loss: 0.7115 - val_accuracy: 0.8047
Epoch 3/20
75/75 [==============================] - 27s 362ms/step - loss: 1.4627 - accuracy: 0.4631 - val_loss: 0.7539 - val_accuracy: 0.8123
Epoch 4/20
75/75 [==============================] - 26s 353ms/step - loss: 1.3832 - accuracy: 0.5088 - val_loss: 0.7332 - val_accuracy: 0.8002
Epoch 5/20
75/75 [==============================] - 27s 361ms/step - loss: 1.3389 - accuracy: 0.5166 - val_loss: 0.5990 - val_accuracy: 0.8284
Epoch 6/20
75/75 [==============================] - 27s 365ms/step - loss: 1.2932 - accuracy: 0.5358 - val_loss: 0.7251 - val_accuracy: 0.7703
Epoch 7/20
75/75 [==============================] - 27s 358ms/step - loss: 1.2092 - accuracy: 0.5543 - val_loss: 0.6373 - val_accuracy: 0.80

In [25]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 37s 310ms/step - loss: 1.6991 - accuracy: 0.3961 - val_loss: 0.8742 - val_accuracy: 0.7343
Epoch 2/20
113/113 [==============================] - 39s 343ms/step - loss: 1.4965 - accuracy: 0.4551 - val_loss: 0.6467 - val_accuracy: 0.7814
Epoch 3/20
113/113 [==============================] - 39s 350ms/step - loss: 1.4299 - accuracy: 0.4842 - val_loss: 0.6772 - val_accuracy: 0.8046
Epoch 4/20
113/113 [==============================] - 39s 347ms/step - loss: 1.3766 - accuracy: 0.4922 - val_loss: 0.7024 - val_accuracy: 0.7859
Epoch 5/20
113/113 [==============================] - 39s 347ms/step - loss: 1.3087 - accuracy: 0.5242 - val_loss: 0.6902 - val_accuracy: 0.8089
Epoch 6/20
113/113 [==============================] - 39s 346ms/step - loss: 1.2481 - accuracy: 0.5434 - val_loss: 0.6536 - val_accuracy: 0.8063
Epoch 7/20
113/113 [==============================] - 39s 346ms/step - loss: 1.2094 - accuracy: 0.5501 - val_loss: 0.5681 - val_

In [26]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 51s 323ms/step - loss: 1.6488 - accuracy: 0.4126 - val_loss: 0.8118 - val_accuracy: 0.7759
Epoch 2/20
150/150 [==============================] - 49s 325ms/step - loss: 1.4605 - accuracy: 0.4685 - val_loss: 0.6826 - val_accuracy: 0.8004
Epoch 3/20
150/150 [==============================] - 48s 322ms/step - loss: 1.3876 - accuracy: 0.4997 - val_loss: 0.7110 - val_accuracy: 0.8064
Epoch 4/20
150/150 [==============================] - 47s 317ms/step - loss: 1.3599 - accuracy: 0.5046 - val_loss: 0.6734 - val_accuracy: 0.7965
Epoch 5/20
150/150 [==============================] - 49s 324ms/step - loss: 1.2770 - accuracy: 0.5382 - val_loss: 0.6687 - val_accuracy: 0.7946
Epoch 6/20
150/150 [==============================] - 48s 320ms/step - loss: 1.2622 - accuracy: 0.5424 - val_loss: 0.5916 - val_accuracy: 0.8083
Epoch 7/20
150/150 [==============================] - 48s 320ms/step - loss: 1.2057 - accuracy: 0.5605 - val_loss: 0.6468 - val_

In [27]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 61s 310ms/step - loss: 1.6414 - accuracy: 0.4307 - val_loss: 0.6041 - val_accuracy: 0.8268
Epoch 2/20
188/188 [==============================] - 58s 310ms/step - loss: 1.4339 - accuracy: 0.4729 - val_loss: 0.6374 - val_accuracy: 0.8171
Epoch 3/20
188/188 [==============================] - 59s 313ms/step - loss: 1.3845 - accuracy: 0.4989 - val_loss: 0.6758 - val_accuracy: 0.8119
Epoch 4/20
188/188 [==============================] - 57s 305ms/step - loss: 1.3285 - accuracy: 0.5206 - val_loss: 0.6455 - val_accuracy: 0.8005
Epoch 5/20
188/188 [==============================] - 58s 308ms/step - loss: 1.2782 - accuracy: 0.5365 - val_loss: 0.6623 - val_accuracy: 0.8035
Epoch 6/20
188/188 [==============================] - 57s 304ms/step - loss: 1.2304 - accuracy: 0.5564 - val_loss: 0.6034 - val_accuracy: 0.8108
Epoch 7/20
188/188 [==============================] - 57s 305ms/step - loss: 1.1947 - accuracy: 0.5729 - val_loss: 0.6374 - val_

In [28]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 68s 291ms/step - loss: 1.5599 - accuracy: 0.4506 - val_loss: 0.7559 - val_accuracy: 0.8307
Epoch 2/20
225/225 [==============================] - 66s 294ms/step - loss: 1.3796 - accuracy: 0.5063 - val_loss: 0.6662 - val_accuracy: 0.8295
Epoch 3/20
225/225 [==============================] - 66s 294ms/step - loss: 1.3345 - accuracy: 0.5133 - val_loss: 0.6679 - val_accuracy: 0.8178
Epoch 4/20
225/225 [==============================] - 66s 294ms/step - loss: 1.2975 - accuracy: 0.5245 - val_loss: 0.6406 - val_accuracy: 0.8131
Epoch 5/20
225/225 [==============================] - 70s 314ms/step - loss: 1.2438 - accuracy: 0.5512 - val_loss: 0.7062 - val_accuracy: 0.8048
Epoch 6/20
225/225 [==============================] - 69s 309ms/step - loss: 1.2029 - accuracy: 0.5687 - val_loss: 0.6820 - val_accuracy: 0.8130
Epoch 7/20
225/225 [==============================] - 71s 314ms/step - loss: 1.1762 - accuracy: 0.5752 - val_loss: 0.6300 - val_

In [29]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 80s 293ms/step - loss: 1.5314 - accuracy: 0.4470 - val_loss: 0.7255 - val_accuracy: 0.8042
Epoch 2/20
263/263 [==============================] - 75s 287ms/step - loss: 1.3675 - accuracy: 0.5051 - val_loss: 0.6070 - val_accuracy: 0.8169
Epoch 3/20
263/263 [==============================] - 75s 287ms/step - loss: 1.3215 - accuracy: 0.5212 - val_loss: 0.5554 - val_accuracy: 0.8213
Epoch 4/20
263/263 [==============================] - 75s 287ms/step - loss: 1.2542 - accuracy: 0.5445 - val_loss: 0.6928 - val_accuracy: 0.8038
Epoch 5/20
263/263 [==============================] - 78s 295ms/step - loss: 1.2259 - accuracy: 0.5572 - val_loss: 0.6436 - val_accuracy: 0.8052
Epoch 6/20
263/263 [==============================] - 76s 288ms/step - loss: 1.1899 - accuracy: 0.5684 - val_loss: 0.6830 - val_accuracy: 0.8056
Epoch 7/20
263/263 [==============================] - 76s 288ms/step - loss: 1.1635 - accuracy: 0.5783 - val_loss: 0.7582 - val_

In [30]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 86s 278ms/step - loss: 1.4919 - accuracy: 0.4766 - val_loss: 0.6727 - val_accuracy: 0.8242
Epoch 2/20
300/300 [==============================] - 84s 281ms/step - loss: 1.3350 - accuracy: 0.5177 - val_loss: 0.6050 - val_accuracy: 0.8070
Epoch 3/20
300/300 [==============================] - 83s 278ms/step - loss: 1.2801 - accuracy: 0.5286 - val_loss: 0.6311 - val_accuracy: 0.8138
Epoch 4/20
300/300 [==============================] - 84s 282ms/step - loss: 1.2233 - accuracy: 0.5590 - val_loss: 0.6513 - val_accuracy: 0.8058
Epoch 5/20
300/300 [==============================] - 84s 280ms/step - loss: 1.1907 - accuracy: 0.5615 - val_loss: 0.6586 - val_accuracy: 0.8018
Epoch 6/20
300/300 [==============================] - 84s 279ms/step - loss: 1.1642 - accuracy: 0.5783 - val_loss: 0.6324 - val_accuracy: 0.8137
Epoch 7/20
300/300 [==============================] - 84s 281ms/step - loss: 1.1332 - accuracy: 0.5914 - val_loss: 0.7276 - val_

In [31]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 96s 277ms/step - loss: 1.4231 - accuracy: 0.5044 - val_loss: 0.7981 - val_accuracy: 0.8098
Epoch 2/20
338/338 [==============================] - 93s 277ms/step - loss: 1.2970 - accuracy: 0.5379 - val_loss: 0.6786 - val_accuracy: 0.8134
Epoch 3/20
338/338 [==============================] - 93s 275ms/step - loss: 1.2360 - accuracy: 0.5503 - val_loss: 0.6586 - val_accuracy: 0.8045
Epoch 4/20
338/338 [==============================] - 93s 275ms/step - loss: 1.1977 - accuracy: 0.5629 - val_loss: 0.7408 - val_accuracy: 0.8003
Epoch 5/20
338/338 [==============================] - 93s 276ms/step - loss: 1.1582 - accuracy: 0.5862 - val_loss: 0.7308 - val_accuracy: 0.7876
Epoch 6/20
338/338 [==============================] - 93s 277ms/step - loss: 1.1315 - accuracy: 0.5880 - val_loss: 0.6364 - val_accuracy: 0.8072
Epoch 7/20
338/338 [==============================] - 94s 277ms/step - loss: 1.1245 - accuracy: 0.5946 - val_loss: 0.6385 - val_

In [32]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 104s 270ms/step - loss: 1.3694 - accuracy: 0.5242 - val_loss: 0.6956 - val_accuracy: 0.8084
Epoch 2/20
375/375 [==============================] - 101s 269ms/step - loss: 1.2313 - accuracy: 0.5533 - val_loss: 0.6305 - val_accuracy: 0.8215
Epoch 3/20
375/375 [==============================] - 101s 270ms/step - loss: 1.1946 - accuracy: 0.5704 - val_loss: 0.6159 - val_accuracy: 0.8123
Epoch 4/20
375/375 [==============================] - 101s 269ms/step - loss: 1.1535 - accuracy: 0.5939 - val_loss: 0.7211 - val_accuracy: 0.8028
Epoch 5/20
375/375 [==============================] - 101s 270ms/step - loss: 1.1365 - accuracy: 0.5880 - val_loss: 0.6166 - val_accuracy: 0.8168
Epoch 6/20
375/375 [==============================] - 101s 271ms/step - loss: 1.0974 - accuracy: 0.6073 - val_loss: 0.5911 - val_accuracy: 0.8128
Epoch 7/20
375/375 [==============================] - 100s 268ms/step - loss: 1.0691 - accuracy: 0.6171 - val_loss: 0.6258

In [33]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 114s 269ms/step - loss: 1.3275 - accuracy: 0.5398 - val_loss: 0.6613 - val_accuracy: 0.7843
Epoch 2/20
413/413 [==============================] - 111s 268ms/step - loss: 1.1861 - accuracy: 0.5760 - val_loss: 0.7025 - val_accuracy: 0.7999
Epoch 3/20
413/413 [==============================] - 112s 271ms/step - loss: 1.1455 - accuracy: 0.5910 - val_loss: 0.6299 - val_accuracy: 0.8019
Epoch 4/20
413/413 [==============================] - 111s 269ms/step - loss: 1.1252 - accuracy: 0.6014 - val_loss: 0.6525 - val_accuracy: 0.8061
Epoch 5/20
413/413 [==============================] - 112s 271ms/step - loss: 1.0883 - accuracy: 0.6098 - val_loss: 0.5882 - val_accuracy: 0.8094
Epoch 6/20
413/413 [==============================] - 112s 271ms/step - loss: 1.0597 - accuracy: 0.6206 - val_loss: 0.6082 - val_accuracy: 0.8053
Epoch 7/20
413/413 [==============================] - 112s 270ms/step - loss: 1.0509 - accuracy: 0.6248 - val_loss: 0.651

In [34]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 121s 264ms/step - loss: 1.2785 - accuracy: 0.5584 - val_loss: 0.6169 - val_accuracy: 0.8223
Epoch 2/20
450/450 [==============================] - 118s 263ms/step - loss: 1.1646 - accuracy: 0.5934 - val_loss: 0.7419 - val_accuracy: 0.7642
Epoch 3/20
450/450 [==============================] - 120s 266ms/step - loss: 1.1188 - accuracy: 0.6020 - val_loss: 0.6564 - val_accuracy: 0.7990
Epoch 4/20
450/450 [==============================] - 121s 269ms/step - loss: 1.0895 - accuracy: 0.6197 - val_loss: 0.6421 - val_accuracy: 0.8096
Epoch 5/20
450/450 [==============================] - 119s 265ms/step - loss: 1.0516 - accuracy: 0.6256 - val_loss: 0.6327 - val_accuracy: 0.7896
Epoch 6/20
450/450 [==============================] - 119s 265ms/step - loss: 1.0320 - accuracy: 0.6362 - val_loss: 0.7260 - val_accuracy: 0.7881
Epoch 7/20
450/450 [==============================] - 120s 266ms/step - loss: 1.0079 - accuracy: 0.6476 - val_loss: 0.767

In [35]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 132s 265ms/step - loss: 1.2242 - accuracy: 0.5824 - val_loss: 0.6799 - val_accuracy: 0.8031
Epoch 2/20
488/488 [==============================] - 130s 266ms/step - loss: 1.1052 - accuracy: 0.6126 - val_loss: 0.6567 - val_accuracy: 0.8101
Epoch 3/20
488/488 [==============================] - 130s 267ms/step - loss: 1.0633 - accuracy: 0.6281 - val_loss: 0.5904 - val_accuracy: 0.8230
Epoch 4/20
488/488 [==============================] - 129s 265ms/step - loss: 1.0440 - accuracy: 0.6354 - val_loss: 0.6486 - val_accuracy: 0.7933
Epoch 5/20
488/488 [==============================] - 130s 266ms/step - loss: 1.0003 - accuracy: 0.6483 - val_loss: 0.5970 - val_accuracy: 0.8125
Epoch 6/20
488/488 [==============================] - 130s 266ms/step - loss: 0.9954 - accuracy: 0.6466 - val_loss: 0.7114 - val_accuracy: 0.7612
Epoch 7/20
488/488 [==============================] - 130s 266ms/step - loss: 0.9794 - accuracy: 0.6603 - val_loss: 0.633

In [36]:

print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 139s 259ms/step - loss: 1.1615 - accuracy: 0.6068 - val_loss: 0.5850 - val_accuracy: 0.8353
Epoch 2/20
525/525 [==============================] - 137s 262ms/step - loss: 1.0590 - accuracy: 0.6355 - val_loss: 0.5641 - val_accuracy: 0.8257
Epoch 3/20
525/525 [==============================] - 145s 277ms/step - loss: 1.0243 - accuracy: 0.6442 - val_loss: 0.5967 - val_accuracy: 0.8065
Epoch 4/20
525/525 [==============================] - 137s 261ms/step - loss: 1.0124 - accuracy: 0.6500 - val_loss: 0.6296 - val_accuracy: 0.8110
Epoch 5/20
525/525 [==============================] - 137s 261ms/step - loss: 0.9707 - accuracy: 0.6600 - val_loss: 0.5670 - val_accuracy: 0.8205
Epoch 6/20
525/525 [==============================] - 136s 260ms/step - loss: 0.9689 - accuracy: 0.6618 - val_loss: 0.5363 - val_accuracy: 0.8228
Epoch 7/20
525/525 [==============================] - 137s 260ms/step - loss: 0.9441 - accuracy: 0.6727 - val_loss: 0.590

In [37]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 150s 262ms/step - loss: 1.1092 - accuracy: 0.6317 - val_loss: 0.6526 - val_accuracy: 0.8173
Epoch 2/20
563/563 [==============================] - 147s 262ms/step - loss: 1.0169 - accuracy: 0.6503 - val_loss: 0.5810 - val_accuracy: 0.8209
Epoch 3/20
563/563 [==============================] - 148s 263ms/step - loss: 0.9749 - accuracy: 0.6587 - val_loss: 0.6712 - val_accuracy: 0.8117
Epoch 4/20
563/563 [==============================] - 148s 262ms/step - loss: 0.9571 - accuracy: 0.6666 - val_loss: 0.7219 - val_accuracy: 0.7706
Epoch 5/20
563/563 [==============================] - 150s 267ms/step - loss: 0.9427 - accuracy: 0.6724 - val_loss: 0.7196 - val_accuracy: 0.8052
Epoch 6/20
563/563 [==============================] - 148s 263ms/step - loss: 0.9199 - accuracy: 0.6770 - val_loss: 0.6562 - val_accuracy: 0.8083
Epoch 7/20
563/563 [==============================] - 148s 262ms/step - loss: 0.9146 - accuracy: 0.6825 - val_loss: 0.590

In [38]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 157s 257ms/step - loss: 1.0693 - accuracy: 0.6460 - val_loss: 0.5538 - val_accuracy: 0.8244
Epoch 2/20
600/600 [==============================] - 155s 258ms/step - loss: 0.9733 - accuracy: 0.6634 - val_loss: 0.6359 - val_accuracy: 0.7977
Epoch 3/20
600/600 [==============================] - 160s 266ms/step - loss: 0.9351 - accuracy: 0.6790 - val_loss: 0.5808 - val_accuracy: 0.8161
Epoch 4/20
600/600 [==============================] - 163s 272ms/step - loss: 0.9233 - accuracy: 0.6802 - val_loss: 0.5666 - val_accuracy: 0.8179
Epoch 5/20
600/600 [==============================] - 163s 271ms/step - loss: 0.9157 - accuracy: 0.6829 - val_loss: 0.5798 - val_accuracy: 0.8130
Epoch 6/20
600/600 [==============================] - 157s 261ms/step - loss: 0.8833 - accuracy: 0.6926 - val_loss: 0.6060 - val_accuracy: 0.8052
Epoch 7/20
600/600 [==============================] - 154s 257ms/step - loss: 0.8798 - accuracy: 0.6969 - val_loss: 0.642

In [39]:
time.sleep(400)

In [40]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 168s 261ms/step - loss: 1.0165 - accuracy: 0.6611 - val_loss: 0.6104 - val_accuracy: 0.8225
Epoch 2/20
638/638 [==============================] - 164s 257ms/step - loss: 0.9237 - accuracy: 0.6782 - val_loss: 0.5778 - val_accuracy: 0.8174
Epoch 3/20
638/638 [==============================] - 165s 258ms/step - loss: 0.9041 - accuracy: 0.6877 - val_loss: 0.6333 - val_accuracy: 0.7871
Epoch 4/20
638/638 [==============================] - 164s 257ms/step - loss: 0.8749 - accuracy: 0.7029 - val_loss: 0.6718 - val_accuracy: 0.7827
Epoch 5/20
638/638 [==============================] - 164s 258ms/step - loss: 0.8764 - accuracy: 0.6949 - val_loss: 0.7225 - val_accuracy: 0.7598
Epoch 6/20
638/638 [==============================] - 164s 258ms/step - loss: 0.8692 - accuracy: 0.7044 - val_loss: 0.6751 - val_accuracy: 0.7799
Epoch 7/20
638/638 [==============================] - 164s 257ms/step - loss: 0.8480 - accuracy: 0.7093 - val_loss: 0.583

In [ ]:
time.sleep(400)

In [41]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 172s 251ms/step - loss: 0.9806 - accuracy: 0.6741 - val_loss: 0.6287 - val_accuracy: 0.8106
Epoch 2/20
675/675 [==============================] - 169s 251ms/step - loss: 0.8846 - accuracy: 0.6987 - val_loss: 0.5741 - val_accuracy: 0.8051
Epoch 3/20
675/675 [==============================] - 170s 251ms/step - loss: 0.8681 - accuracy: 0.7012 - val_loss: 0.5216 - val_accuracy: 0.8342
Epoch 4/20
675/675 [==============================] - 170s 252ms/step - loss: 0.8289 - accuracy: 0.7164 - val_loss: 0.5440 - val_accuracy: 0.8271
Epoch 5/20
675/675 [==============================] - 172s 255ms/step - loss: 0.8330 - accuracy: 0.7119 - val_loss: 0.6383 - val_accuracy: 0.8141
Epoch 6/20
675/675 [==============================] - 174s 258ms/step - loss: 0.8254 - accuracy: 0.7110 - val_loss: 0.5945 - val_accuracy: 0.8074
Epoch 7/20
675/675 [==============================] - 171s 253ms/step - loss: 0.8180 - accuracy: 0.7204 - val_loss: 0.557

In [ ]:
time.sleep(400)

In [23]:
n=18
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 241s 304ms/step - loss: 0.9297 - accuracy: 0.6901 - val_loss: 0.6153 - val_accuracy: 0.8225
Epoch 2/20
713/713 [==============================] - 203s 285ms/step - loss: 0.8609 - accuracy: 0.7048 - val_loss: 0.7061 - val_accuracy: 0.7864
Epoch 3/20
713/713 [==============================] - 204s 286ms/step - loss: 0.8394 - accuracy: 0.7142 - val_loss: 0.5880 - val_accuracy: 0.8161
Epoch 4/20
713/713 [==============================] - 208s 292ms/step - loss: 0.8114 - accuracy: 0.7212 - val_loss: 0.6549 - val_accuracy: 0.7966
Epoch 5/20
713/713 [==============================] - 212s 297ms/step - loss: 0.8032 - accuracy: 0.7226 - val_loss: 0.6985 - val_accuracy: 0.7772
Epoch 6/20
713/713 [==============================] - 218s 305ms/step - loss: 0.8021 - accuracy: 0.7277 - val_loss: 0.6236 - val_accuracy: 0.8026
Epoch 7/20
713/713 [==============================] - 220s 309ms/step - loss: 0.7841 - accuracy: 0.7311 - val_loss: 0.669

In [ ]:
time.sleep(400)

In [24]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 209s 272ms/step - loss: 0.8820 - accuracy: 0.7053 - val_loss: 0.5754 - val_accuracy: 0.8144
Epoch 2/20
750/750 [==============================] - 203s 270ms/step - loss: 0.8238 - accuracy: 0.7203 - val_loss: 0.6493 - val_accuracy: 0.8159
Epoch 3/20
750/750 [==============================] - 203s 271ms/step - loss: 0.8005 - accuracy: 0.7276 - val_loss: 0.5919 - val_accuracy: 0.8018
Epoch 4/20
750/750 [==============================] - 201s 268ms/step - loss: 0.7854 - accuracy: 0.7349 - val_loss: 0.5671 - val_accuracy: 0.8238
Epoch 5/20
750/750 [==============================] - 201s 267ms/step - loss: 0.7657 - accuracy: 0.7369 - val_loss: 0.6088 - val_accuracy: 0.8098
Epoch 6/20
750/750 [==============================] - 200s 267ms/step - loss: 0.7543 - accuracy: 0.7436 - val_loss: 0.6015 - val_accuracy: 0.7996
Epoch 7/20
750/750 [==============================] - 199s 265ms/step - loss: 0.7542 - accuracy: 0.7408 - val_loss: 0.634

In [ ]:
len(models_se)

In [25]:
new_model_se_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_se_e1"

i=18

for model in models_se[18:]:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_se_e1_18\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_se_e1_19\assets
Model has been saved


In [26]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [27]:
gc.collect()

420916

## Training guided by Random values

In [ ]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

In [19]:
save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)

In [20]:
len(random_indexes)

47999

In [21]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_train_and_adversary)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_train_and_adversary)

In [22]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [23]:
len(image_sets_random)

20

In [24]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [34]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 26s 602ms/step - loss: 0.9882 - accuracy: 0.6996 - val_loss: 0.6037 - val_accuracy: 0.8277
Epoch 2/20
38/38 [==============================] - 22s 577ms/step - loss: 0.7936 - accuracy: 0.7334 - val_loss: 0.6097 - val_accuracy: 0.8154
Epoch 3/20
38/38 [==============================] - 22s 584ms/step - loss: 0.7541 - accuracy: 0.7301 - val_loss: 0.5293 - val_accuracy: 0.8238
Epoch 4/20
38/38 [==============================] - 22s 600ms/step - loss: 0.6456 - accuracy: 0.7736 - val_loss: 0.5279 - val_accuracy: 0.8190
Epoch 5/20
38/38 [==============================] - 23s 625ms/step - loss: 0.7027 - accuracy: 0.7639 - val_loss: 0.5287 - val_accuracy: 0.8217
Epoch 6/20
38/38 [==============================] - 23s 627ms/step - loss: 0.5932 - accuracy: 0.7937 - val_loss: 0.5273 - val_accuracy: 0.8273
Epoch 7/20
38/38 [==============================] - 23s 628ms/step - loss: 0.5350 - accuracy: 0.8075 - val_loss: 0.5747 - val_accuracy: 0.82

In [35]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 35s 428ms/step - loss: 1.0154 - accuracy: 0.6912 - val_loss: 0.5867 - val_accuracy: 0.8172
Epoch 2/20
75/75 [==============================] - 31s 411ms/step - loss: 0.7991 - accuracy: 0.7285 - val_loss: 0.5391 - val_accuracy: 0.8269
Epoch 3/20
75/75 [==============================] - 31s 411ms/step - loss: 0.7287 - accuracy: 0.7426 - val_loss: 0.5677 - val_accuracy: 0.8246
Epoch 4/20
75/75 [==============================] - 30s 409ms/step - loss: 0.7261 - accuracy: 0.7525 - val_loss: 0.5677 - val_accuracy: 0.8220
Epoch 5/20
75/75 [==============================] - 31s 409ms/step - loss: 0.6840 - accuracy: 0.7629 - val_loss: 0.5307 - val_accuracy: 0.8261
Epoch 6/20
75/75 [==============================] - 31s 411ms/step - loss: 0.6436 - accuracy: 0.7765 - val_loss: 0.5981 - val_accuracy: 0.7981
Epoch 7/20
75/75 [==============================] - 31s 419ms/step - loss: 0.5977 - accuracy: 0.7903 - val_loss: 0.5406 - val_accuracy: 0.81

In [36]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 44s 359ms/step - loss: 0.9694 - accuracy: 0.6977 - val_loss: 0.6110 - val_accuracy: 0.7997
Epoch 2/20
113/113 [==============================] - 40s 359ms/step - loss: 0.8162 - accuracy: 0.7189 - val_loss: 0.6039 - val_accuracy: 0.8092
Epoch 3/20
113/113 [==============================] - 42s 372ms/step - loss: 0.7699 - accuracy: 0.7420 - val_loss: 0.5827 - val_accuracy: 0.8150
Epoch 4/20
113/113 [==============================] - 42s 375ms/step - loss: 0.7459 - accuracy: 0.7431 - val_loss: 0.5417 - val_accuracy: 0.8251
Epoch 5/20
113/113 [==============================] - 40s 357ms/step - loss: 0.7236 - accuracy: 0.7562 - val_loss: 0.6338 - val_accuracy: 0.7975
Epoch 6/20
113/113 [==============================] - 40s 358ms/step - loss: 0.6732 - accuracy: 0.7662 - val_loss: 0.6342 - val_accuracy: 0.7991
Epoch 7/20
113/113 [==============================] - 40s 356ms/step - loss: 0.6790 - accuracy: 0.7652 - val_loss: 0.5613 - val_

In [37]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 53s 328ms/step - loss: 0.9170 - accuracy: 0.7111 - val_loss: 0.5520 - val_accuracy: 0.8291
Epoch 2/20
150/150 [==============================] - 49s 330ms/step - loss: 0.8083 - accuracy: 0.7269 - val_loss: 0.6603 - val_accuracy: 0.8082
Epoch 3/20
150/150 [==============================] - 49s 329ms/step - loss: 0.7757 - accuracy: 0.7369 - val_loss: 0.5843 - val_accuracy: 0.8276
Epoch 4/20
150/150 [==============================] - 50s 337ms/step - loss: 0.7196 - accuracy: 0.7576 - val_loss: 0.5297 - val_accuracy: 0.8292
Epoch 5/20
150/150 [==============================] - 49s 330ms/step - loss: 0.6885 - accuracy: 0.7596 - val_loss: 0.5879 - val_accuracy: 0.8227
Epoch 6/20
150/150 [==============================] - 49s 330ms/step - loss: 0.6906 - accuracy: 0.7669 - val_loss: 0.6146 - val_accuracy: 0.8026
Epoch 7/20
150/150 [==============================] - 50s 334ms/step - loss: 0.6673 - accuracy: 0.7683 - val_loss: 0.5812 - val_

In [25]:
n=4
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 72s 319ms/step - loss: 0.9258 - accuracy: 0.6959 - val_loss: 0.6007 - val_accuracy: 0.8170
Epoch 2/20
188/188 [==============================] - 53s 283ms/step - loss: 0.8126 - accuracy: 0.7229 - val_loss: 0.6204 - val_accuracy: 0.8201
Epoch 3/20
188/188 [==============================] - 49s 261ms/step - loss: 0.7686 - accuracy: 0.7315 - val_loss: 0.5855 - val_accuracy: 0.8200
Epoch 4/20
188/188 [==============================] - 45s 238ms/step - loss: 0.7442 - accuracy: 0.7449 - val_loss: 0.7149 - val_accuracy: 0.7727
Epoch 5/20
188/188 [==============================] - 45s 240ms/step - loss: 0.7229 - accuracy: 0.7484 - val_loss: 0.5782 - val_accuracy: 0.8083
Epoch 6/20
188/188 [==============================] - 45s 239ms/step - loss: 0.6889 - accuracy: 0.7633 - val_loss: 0.5669 - val_accuracy: 0.8150
Epoch 7/20
188/188 [==============================] - 45s 239ms/step - loss: 0.6771 - accuracy: 0.7715 - val_loss: 0.5991 - val_

In [26]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 53s 228ms/step - loss: 0.9405 - accuracy: 0.7002 - val_loss: 0.6289 - val_accuracy: 0.8205
Epoch 2/20
225/225 [==============================] - 52s 230ms/step - loss: 0.8064 - accuracy: 0.7270 - val_loss: 0.5927 - val_accuracy: 0.8174
Epoch 3/20
225/225 [==============================] - 57s 253ms/step - loss: 0.7763 - accuracy: 0.7343 - val_loss: 0.6338 - val_accuracy: 0.8190
Epoch 4/20
225/225 [==============================] - 58s 256ms/step - loss: 0.7513 - accuracy: 0.7474 - val_loss: 0.5550 - val_accuracy: 0.8148
Epoch 5/20
225/225 [==============================] - 58s 258ms/step - loss: 0.7411 - accuracy: 0.7453 - val_loss: 0.5412 - val_accuracy: 0.8292
Epoch 6/20
225/225 [==============================] - 57s 255ms/step - loss: 0.7121 - accuracy: 0.7497 - val_loss: 0.6579 - val_accuracy: 0.8030
Epoch 7/20
225/225 [==============================] - 57s 256ms/step - loss: 0.6876 - accuracy: 0.7601 - val_loss: 0.5902 - val_

In [27]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 68s 252ms/step - loss: 0.9489 - accuracy: 0.6930 - val_loss: 0.6841 - val_accuracy: 0.7770
Epoch 2/20
263/263 [==============================] - 66s 251ms/step - loss: 0.8246 - accuracy: 0.7212 - val_loss: 0.5424 - val_accuracy: 0.8277
Epoch 3/20
263/263 [==============================] - 66s 251ms/step - loss: 0.7915 - accuracy: 0.7327 - val_loss: 0.6456 - val_accuracy: 0.7928
Epoch 4/20
263/263 [==============================] - 66s 251ms/step - loss: 0.7477 - accuracy: 0.7392 - val_loss: 0.5447 - val_accuracy: 0.8267
Epoch 5/20
263/263 [==============================] - 66s 251ms/step - loss: 0.7266 - accuracy: 0.7526 - val_loss: 0.6778 - val_accuracy: 0.7835
Epoch 6/20
263/263 [==============================] - 67s 255ms/step - loss: 0.7309 - accuracy: 0.7455 - val_loss: 0.5626 - val_accuracy: 0.8156
Epoch 7/20
263/263 [==============================] - 66s 251ms/step - loss: 0.6883 - accuracy: 0.7618 - val_loss: 0.6440 - val_

In [28]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 76s 246ms/step - loss: 0.9287 - accuracy: 0.7002 - val_loss: 0.6769 - val_accuracy: 0.8134
Epoch 2/20
300/300 [==============================] - 77s 258ms/step - loss: 0.8279 - accuracy: 0.7163 - val_loss: 0.6007 - val_accuracy: 0.7994
Epoch 3/20
300/300 [==============================] - 74s 248ms/step - loss: 0.7941 - accuracy: 0.7309 - val_loss: 0.5564 - val_accuracy: 0.8163
Epoch 4/20
300/300 [==============================] - 74s 246ms/step - loss: 0.7484 - accuracy: 0.7439 - val_loss: 0.6013 - val_accuracy: 0.8195
Epoch 5/20
300/300 [==============================] - 75s 249ms/step - loss: 0.7353 - accuracy: 0.7454 - val_loss: 0.5618 - val_accuracy: 0.8143
Epoch 6/20
300/300 [==============================] - 78s 260ms/step - loss: 0.7370 - accuracy: 0.7501 - val_loss: 0.5836 - val_accuracy: 0.8137
Epoch 7/20
300/300 [==============================] - 78s 261ms/step - loss: 0.6958 - accuracy: 0.7593 - val_loss: 0.6127 - val_

In [29]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 85s 245ms/step - loss: 0.9137 - accuracy: 0.7035 - val_loss: 0.5810 - val_accuracy: 0.8197
Epoch 2/20
338/338 [==============================] - 83s 245ms/step - loss: 0.8189 - accuracy: 0.7296 - val_loss: 0.6236 - val_accuracy: 0.8035
Epoch 3/20
338/338 [==============================] - 83s 245ms/step - loss: 0.7919 - accuracy: 0.7287 - val_loss: 0.6676 - val_accuracy: 0.7960
Epoch 4/20
338/338 [==============================] - 83s 247ms/step - loss: 0.7636 - accuracy: 0.7361 - val_loss: 0.5967 - val_accuracy: 0.8002
Epoch 5/20
338/338 [==============================] - 84s 247ms/step - loss: 0.7488 - accuracy: 0.7395 - val_loss: 0.7392 - val_accuracy: 0.7804
Epoch 6/20
338/338 [==============================] - 83s 246ms/step - loss: 0.7172 - accuracy: 0.7535 - val_loss: 0.8218 - val_accuracy: 0.7347
Epoch 7/20
338/338 [==============================] - 85s 250ms/step - loss: 0.7086 - accuracy: 0.7549 - val_loss: 0.5638 - val_

In [30]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 92s 238ms/step - loss: 0.9069 - accuracy: 0.7017 - val_loss: 0.5935 - val_accuracy: 0.8303
Epoch 2/20
375/375 [==============================] - 89s 238ms/step - loss: 0.8166 - accuracy: 0.7247 - val_loss: 0.5782 - val_accuracy: 0.8241
Epoch 3/20
375/375 [==============================] - 90s 239ms/step - loss: 0.7906 - accuracy: 0.7313 - val_loss: 0.6034 - val_accuracy: 0.8180
Epoch 4/20
375/375 [==============================] - 91s 242ms/step - loss: 0.7561 - accuracy: 0.7379 - val_loss: 0.5478 - val_accuracy: 0.8223
Epoch 5/20
375/375 [==============================] - 89s 238ms/step - loss: 0.7415 - accuracy: 0.7408 - val_loss: 0.5807 - val_accuracy: 0.8152
Epoch 6/20
375/375 [==============================] - 90s 239ms/step - loss: 0.7478 - accuracy: 0.7366 - val_loss: 0.5524 - val_accuracy: 0.8196
Epoch 7/20
375/375 [==============================] - 89s 238ms/step - loss: 0.7167 - accuracy: 0.7509 - val_loss: 0.6789 - val_

In [31]:
n=10
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 103s 242ms/step - loss: 0.9242 - accuracy: 0.6995 - val_loss: 0.7299 - val_accuracy: 0.8105
Epoch 2/20
413/413 [==============================] - 99s 239ms/step - loss: 0.8091 - accuracy: 0.7271 - val_loss: 0.6484 - val_accuracy: 0.8163
Epoch 3/20
413/413 [==============================] - 98s 239ms/step - loss: 0.7934 - accuracy: 0.7304 - val_loss: 0.7060 - val_accuracy: 0.7705
Epoch 4/20
413/413 [==============================] - 99s 241ms/step - loss: 0.7604 - accuracy: 0.7358 - val_loss: 0.6984 - val_accuracy: 0.7754
Epoch 5/20
413/413 [==============================] - 100s 243ms/step - loss: 0.7608 - accuracy: 0.7402 - val_loss: 0.5603 - val_accuracy: 0.8202
Epoch 6/20
413/413 [==============================] - 100s 241ms/step - loss: 0.7447 - accuracy: 0.7458 - val_loss: 0.6648 - val_accuracy: 0.7994
Epoch 7/20
413/413 [==============================] - 98s 238ms/step - loss: 0.7329 - accuracy: 0.7493 - val_loss: 0.6667 - 

In [32]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 109s 236ms/step - loss: 0.9186 - accuracy: 0.7017 - val_loss: 0.7840 - val_accuracy: 0.7904
Epoch 2/20
450/450 [==============================] - 106s 235ms/step - loss: 0.8132 - accuracy: 0.7199 - val_loss: 0.6094 - val_accuracy: 0.8025
Epoch 3/20
450/450 [==============================] - 104s 230ms/step - loss: 0.7772 - accuracy: 0.7356 - val_loss: 0.5847 - val_accuracy: 0.8099
Epoch 4/20
450/450 [==============================] - 105s 233ms/step - loss: 0.7684 - accuracy: 0.7368 - val_loss: 0.5866 - val_accuracy: 0.8146
Epoch 5/20
450/450 [==============================] - 105s 234ms/step - loss: 0.7428 - accuracy: 0.7474 - val_loss: 0.5812 - val_accuracy: 0.8107
Epoch 6/20
450/450 [==============================] - 105s 234ms/step - loss: 0.7421 - accuracy: 0.7465 - val_loss: 0.7304 - val_accuracy: 0.7839
Epoch 7/20
450/450 [==============================] - 105s 233ms/step - loss: 0.7300 - accuracy: 0.7536 - val_loss: 0.606

In [33]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 124s 249ms/step - loss: 0.9157 - accuracy: 0.7002 - val_loss: 0.7200 - val_accuracy: 0.7840
Epoch 2/20
488/488 [==============================] - 121s 247ms/step - loss: 0.8201 - accuracy: 0.7240 - val_loss: 0.6059 - val_accuracy: 0.8072
Epoch 3/20
488/488 [==============================] - 115s 235ms/step - loss: 0.7949 - accuracy: 0.7288 - val_loss: 0.5797 - val_accuracy: 0.8051
Epoch 4/20
488/488 [==============================] - 114s 234ms/step - loss: 0.7832 - accuracy: 0.7314 - val_loss: 0.8357 - val_accuracy: 0.7513
Epoch 5/20
488/488 [==============================] - 114s 234ms/step - loss: 0.7551 - accuracy: 0.7394 - val_loss: 0.5869 - val_accuracy: 0.8180
Epoch 6/20
488/488 [==============================] - 115s 236ms/step - loss: 0.7509 - accuracy: 0.7413 - val_loss: 0.6664 - val_accuracy: 0.8012
Epoch 7/20
488/488 [==============================] - 114s 234ms/step - loss: 0.7339 - accuracy: 0.7498 - val_loss: 0.760

In [34]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 123s 229ms/step - loss: 0.8949 - accuracy: 0.7028 - val_loss: 0.6275 - val_accuracy: 0.8236
Epoch 2/20
525/525 [==============================] - 120s 228ms/step - loss: 0.8234 - accuracy: 0.7254 - val_loss: 0.6591 - val_accuracy: 0.8027
Epoch 3/20
525/525 [==============================] - 120s 229ms/step - loss: 0.7852 - accuracy: 0.7344 - val_loss: 0.5906 - val_accuracy: 0.8209
Epoch 4/20
525/525 [==============================] - 120s 229ms/step - loss: 0.7765 - accuracy: 0.7346 - val_loss: 0.5939 - val_accuracy: 0.8075
Epoch 5/20
525/525 [==============================] - 121s 231ms/step - loss: 0.7552 - accuracy: 0.7421 - val_loss: 0.5990 - val_accuracy: 0.8022
Epoch 6/20
525/525 [==============================] - 121s 231ms/step - loss: 0.7509 - accuracy: 0.7453 - val_loss: 0.5441 - val_accuracy: 0.8142
Epoch 7/20
525/525 [==============================] - 120s 229ms/step - loss: 0.7337 - accuracy: 0.7452 - val_loss: 0.603

In [35]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 132s 230ms/step - loss: 0.8976 - accuracy: 0.7035 - val_loss: 0.5469 - val_accuracy: 0.8242
Epoch 2/20
563/563 [==============================] - 130s 231ms/step - loss: 0.8297 - accuracy: 0.7172 - val_loss: 0.6071 - val_accuracy: 0.8247
Epoch 3/20
563/563 [==============================] - 130s 230ms/step - loss: 0.7926 - accuracy: 0.7291 - val_loss: 0.6889 - val_accuracy: 0.8040
Epoch 4/20
563/563 [==============================] - 131s 233ms/step - loss: 0.7774 - accuracy: 0.7342 - val_loss: 0.6189 - val_accuracy: 0.7958
Epoch 5/20
563/563 [==============================] - 130s 231ms/step - loss: 0.7623 - accuracy: 0.7380 - val_loss: 0.6746 - val_accuracy: 0.7918
Epoch 6/20
563/563 [==============================] - 130s 232ms/step - loss: 0.7476 - accuracy: 0.7435 - val_loss: 0.5739 - val_accuracy: 0.8097
Epoch 7/20
563/563 [==============================] - 130s 231ms/step - loss: 0.7433 - accuracy: 0.7470 - val_loss: 0.606

In [36]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 140s 229ms/step - loss: 0.9092 - accuracy: 0.7007 - val_loss: 0.6124 - val_accuracy: 0.8207
Epoch 2/20
600/600 [==============================] - 136s 226ms/step - loss: 0.8136 - accuracy: 0.7227 - val_loss: 0.6372 - val_accuracy: 0.7916
Epoch 3/20
600/600 [==============================] - 138s 230ms/step - loss: 0.7923 - accuracy: 0.7329 - val_loss: 0.6210 - val_accuracy: 0.8019
Epoch 4/20
600/600 [==============================] - 138s 231ms/step - loss: 0.7834 - accuracy: 0.7340 - val_loss: 0.5804 - val_accuracy: 0.8192
Epoch 5/20
600/600 [==============================] - 137s 228ms/step - loss: 0.7617 - accuracy: 0.7396 - val_loss: 0.5769 - val_accuracy: 0.8122
Epoch 6/20
600/600 [==============================] - 136s 228ms/step - loss: 0.7474 - accuracy: 0.7444 - val_loss: 0.7149 - val_accuracy: 0.7976
Epoch 7/20
600/600 [==============================] - 137s 228ms/step - loss: 0.7554 - accuracy: 0.7413 - val_loss: 0.761

In [37]:
time.sleep(400)

In [38]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 151s 234ms/step - loss: 0.8955 - accuracy: 0.7042 - val_loss: 0.6731 - val_accuracy: 0.7770
Epoch 2/20
638/638 [==============================] - 146s 228ms/step - loss: 0.8165 - accuracy: 0.7221 - val_loss: 0.5786 - val_accuracy: 0.8131
Epoch 3/20
638/638 [==============================] - 172s 269ms/step - loss: 0.7975 - accuracy: 0.7271 - val_loss: 0.7261 - val_accuracy: 0.7733
Epoch 4/20
638/638 [==============================] - 203s 318ms/step - loss: 0.7699 - accuracy: 0.7375 - val_loss: 0.5890 - val_accuracy: 0.8133
Epoch 5/20
638/638 [==============================] - 208s 327ms/step - loss: 0.7682 - accuracy: 0.7335 - val_loss: 0.5768 - val_accuracy: 0.8090
Epoch 6/20
638/638 [==============================] - 190s 298ms/step - loss: 0.7628 - accuracy: 0.7396 - val_loss: 0.6622 - val_accuracy: 0.7860
Epoch 7/20
638/638 [==============================] - 181s 283ms/step - loss: 0.7556 - accuracy: 0.7439 - val_loss: 0.950

In [ ]:
time.sleep(400)

In [39]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 159s 232ms/step - loss: 0.8879 - accuracy: 0.7055 - val_loss: 0.6073 - val_accuracy: 0.7978
Epoch 2/20
675/675 [==============================] - 163s 241ms/step - loss: 0.8261 - accuracy: 0.7189 - val_loss: 0.5991 - val_accuracy: 0.8114
Epoch 3/20
675/675 [==============================] - 164s 244ms/step - loss: 0.7944 - accuracy: 0.7291 - val_loss: 0.5843 - val_accuracy: 0.8188
Epoch 4/20
675/675 [==============================] - 187s 277ms/step - loss: 0.7881 - accuracy: 0.7281 - val_loss: 0.5576 - val_accuracy: 0.8127
Epoch 5/20
675/675 [==============================] - 198s 294ms/step - loss: 0.7716 - accuracy: 0.7347 - val_loss: 0.6062 - val_accuracy: 0.7962
Epoch 6/20
675/675 [==============================] - 196s 290ms/step - loss: 0.7669 - accuracy: 0.7392 - val_loss: 0.6112 - val_accuracy: 0.8033
Epoch 7/20
675/675 [==============================] - 202s 300ms/step - loss: 0.7442 - accuracy: 0.7458 - val_loss: 0.580

In [ ]:
time.sleep(400)

In [40]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 168s 232ms/step - loss: 0.8900 - accuracy: 0.7041 - val_loss: 0.5727 - val_accuracy: 0.8111
Epoch 2/20
713/713 [==============================] - 167s 234ms/step - loss: 0.8312 - accuracy: 0.7191 - val_loss: 0.5963 - val_accuracy: 0.8002
Epoch 3/20
713/713 [==============================] - 164s 231ms/step - loss: 0.7802 - accuracy: 0.7317 - val_loss: 0.6266 - val_accuracy: 0.7957
Epoch 4/20
713/713 [==============================] - 165s 231ms/step - loss: 0.7767 - accuracy: 0.7352 - val_loss: 0.6494 - val_accuracy: 0.8031
Epoch 5/20
713/713 [==============================] - 165s 231ms/step - loss: 0.7702 - accuracy: 0.7377 - val_loss: 0.6028 - val_accuracy: 0.8003
Epoch 6/20
713/713 [==============================] - 165s 232ms/step - loss: 0.7499 - accuracy: 0.7405 - val_loss: 0.5821 - val_accuracy: 0.8103
Epoch 7/20
713/713 [==============================] - 165s 232ms/step - loss: 0.7621 - accuracy: 0.7393 - val_loss: 0.696

In [ ]:
time.sleep(400)

In [41]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 188s 247ms/step - loss: 0.8823 - accuracy: 0.7083 - val_loss: 0.5637 - val_accuracy: 0.8272
Epoch 2/20
750/750 [==============================] - 180s 240ms/step - loss: 0.8175 - accuracy: 0.7239 - val_loss: 0.5785 - val_accuracy: 0.8183
Epoch 3/20
750/750 [==============================] - 182s 243ms/step - loss: 0.7879 - accuracy: 0.7299 - val_loss: 0.6547 - val_accuracy: 0.8014
Epoch 4/20
750/750 [==============================] - 181s 241ms/step - loss: 0.7828 - accuracy: 0.7301 - val_loss: 0.6451 - val_accuracy: 0.8083
Epoch 5/20
750/750 [==============================] - 181s 241ms/step - loss: 0.7835 - accuracy: 0.7328 - val_loss: 0.6587 - val_accuracy: 0.8028
Epoch 6/20
750/750 [==============================] - 181s 242ms/step - loss: 0.7716 - accuracy: 0.7380 - val_loss: 0.5835 - val_accuracy: 0.8098
Epoch 7/20
750/750 [==============================] - 183s 244ms/step - loss: 0.7600 - accuracy: 0.7370 - val_loss: 0.605

In [42]:
new_model_random_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_random_e1"

i=4

for model in models_random[4:]:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_random_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_random_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_random_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_random_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_random_e1_8\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_random_e1_9\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_random_e1_10\assets
Model has been saved
INFO:

In [ ]:
loading = False

models_random = []

if loading:
    for i in range(20):
        model_random_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_"+str(i)
        print(model_random_dir)
        model =utils.My_model('gtsrb',True,model_random_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_random.append(model)
       

In [43]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [44]:
gc.collect()

534134

## Training guided by NC

In [21]:
# NC
nc_values = []
for i in range(1,17):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [22]:
nc_values.shape

(47999,)

In [23]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_train_and_adversary)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_train_and_adversary)


In [24]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [25]:
print ("sas")

sas


In [26]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [27]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 44s 570ms/step - loss: 0.4133 - accuracy: 0.8916 - val_loss: 0.6428 - val_accuracy: 0.7819
Epoch 2/20
38/38 [==============================] - 20s 532ms/step - loss: 0.2476 - accuracy: 0.9265 - val_loss: 0.7029 - val_accuracy: 0.7594
Epoch 3/20
38/38 [==============================] - 20s 530ms/step - loss: 0.2479 - accuracy: 0.9148 - val_loss: 0.9956 - val_accuracy: 0.6728
Epoch 4/20
38/38 [==============================] - 20s 523ms/step - loss: 0.2133 - accuracy: 0.9300 - val_loss: 0.7190 - val_accuracy: 0.7766
Epoch 5/20
38/38 [==============================] - 20s 543ms/step - loss: 0.2070 - accuracy: 0.9308 - val_loss: 1.0044 - val_accuracy: 0.7243
Epoch 6/20
38/38 [==============================] - 20s 544ms/step - loss: 0.1715 - accuracy: 0.9433 - val_loss: 0.8120 - val_accuracy: 0.7555
Epoch 7/20
38/38 [==============================] - 21s 548ms/step - loss: 0.1597 - accuracy: 0.9517 - val_loss: 0.7261 - val_accuracy: 0.77

In [28]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 25s 306ms/step - loss: 0.4644 - accuracy: 0.8775 - val_loss: 0.6696 - val_accuracy: 0.7886
Epoch 2/20
75/75 [==============================] - 22s 299ms/step - loss: 0.3504 - accuracy: 0.8866 - val_loss: 0.7624 - val_accuracy: 0.7663
Epoch 3/20
75/75 [==============================] - 23s 307ms/step - loss: 0.3178 - accuracy: 0.8966 - val_loss: 0.6941 - val_accuracy: 0.7697
Epoch 4/20
75/75 [==============================] - 23s 302ms/step - loss: 0.2862 - accuracy: 0.9038 - val_loss: 0.8101 - val_accuracy: 0.7618
Epoch 5/20
75/75 [==============================] - 23s 302ms/step - loss: 0.2908 - accuracy: 0.8999 - val_loss: 0.7946 - val_accuracy: 0.7630
Epoch 6/20
75/75 [==============================] - 25s 333ms/step - loss: 0.2712 - accuracy: 0.9098 - val_loss: 0.7115 - val_accuracy: 0.7832
Epoch 7/20
75/75 [==============================] - 25s 341ms/step - loss: 0.2563 - accuracy: 0.9196 - val_loss: 0.7789 - val_accuracy: 0.74

In [29]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 32s 269ms/step - loss: 0.5427 - accuracy: 0.8573 - val_loss: 0.6321 - val_accuracy: 0.7875
Epoch 2/20
113/113 [==============================] - 30s 267ms/step - loss: 0.3865 - accuracy: 0.8793 - val_loss: 0.6789 - val_accuracy: 0.7691
Epoch 3/20
113/113 [==============================] - 31s 271ms/step - loss: 0.3382 - accuracy: 0.8913 - val_loss: 0.7828 - val_accuracy: 0.7619
Epoch 4/20
113/113 [==============================] - 30s 269ms/step - loss: 0.3324 - accuracy: 0.8896 - val_loss: 0.8760 - val_accuracy: 0.7160
Epoch 5/20
113/113 [==============================] - 30s 267ms/step - loss: 0.3466 - accuracy: 0.8832 - val_loss: 1.1681 - val_accuracy: 0.7371
Epoch 6/20
113/113 [==============================] - 30s 269ms/step - loss: 0.3049 - accuracy: 0.8963 - val_loss: 0.8694 - val_accuracy: 0.7137
Epoch 7/20
113/113 [==============================] - 30s 270ms/step - loss: 0.2853 - accuracy: 0.9077 - val_loss: 0.7222 - val_

In [30]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 44s 279ms/step - loss: 0.5783 - accuracy: 0.8310 - val_loss: 0.8188 - val_accuracy: 0.7325
Epoch 2/20
150/150 [==============================] - 42s 278ms/step - loss: 0.4663 - accuracy: 0.8484 - val_loss: 0.6091 - val_accuracy: 0.7994
Epoch 3/20
150/150 [==============================] - 42s 279ms/step - loss: 0.4292 - accuracy: 0.8591 - val_loss: 0.6577 - val_accuracy: 0.7824
Epoch 4/20
150/150 [==============================] - 41s 277ms/step - loss: 0.4251 - accuracy: 0.8592 - val_loss: 0.6966 - val_accuracy: 0.7679
Epoch 5/20
150/150 [==============================] - 42s 279ms/step - loss: 0.4050 - accuracy: 0.8641 - val_loss: 1.4401 - val_accuracy: 0.6621
Epoch 6/20
150/150 [==============================] - 41s 276ms/step - loss: 0.4064 - accuracy: 0.8634 - val_loss: 0.7480 - val_accuracy: 0.7621
Epoch 7/20
150/150 [==============================] - 42s 282ms/step - loss: 0.3680 - accuracy: 0.8714 - val_loss: 0.7149 - val_

In [31]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 52s 266ms/step - loss: 0.6283 - accuracy: 0.8108 - val_loss: 0.6788 - val_accuracy: 0.7768
Epoch 2/20
188/188 [==============================] - 50s 268ms/step - loss: 0.5248 - accuracy: 0.8269 - val_loss: 0.6841 - val_accuracy: 0.7764
Epoch 3/20
188/188 [==============================] - 50s 265ms/step - loss: 0.5062 - accuracy: 0.8349 - val_loss: 0.6762 - val_accuracy: 0.7811
Epoch 4/20
188/188 [==============================] - 50s 264ms/step - loss: 0.4510 - accuracy: 0.8476 - val_loss: 0.6853 - val_accuracy: 0.7722
Epoch 5/20
188/188 [==============================] - 50s 268ms/step - loss: 0.4483 - accuracy: 0.8558 - val_loss: 0.7697 - val_accuracy: 0.7390
Epoch 6/20
188/188 [==============================] - 50s 264ms/step - loss: 0.4550 - accuracy: 0.8536 - val_loss: 0.7076 - val_accuracy: 0.7740
Epoch 7/20
188/188 [==============================] - 50s 265ms/step - loss: 0.4135 - accuracy: 0.8647 - val_loss: 0.7508 - val_

In [32]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 60s 255ms/step - loss: 0.6233 - accuracy: 0.8101 - val_loss: 0.5974 - val_accuracy: 0.8012
Epoch 2/20
225/225 [==============================] - 57s 256ms/step - loss: 0.5382 - accuracy: 0.8217 - val_loss: 0.6666 - val_accuracy: 0.7799
Epoch 3/20
225/225 [==============================] - 58s 258ms/step - loss: 0.5265 - accuracy: 0.8304 - val_loss: 0.6152 - val_accuracy: 0.7920
Epoch 4/20
225/225 [==============================] - 57s 253ms/step - loss: 0.5029 - accuracy: 0.8324 - val_loss: 0.7182 - val_accuracy: 0.7533
Epoch 5/20
225/225 [==============================] - 55s 246ms/step - loss: 0.4803 - accuracy: 0.8392 - val_loss: 0.7624 - val_accuracy: 0.7463
Epoch 6/20
225/225 [==============================] - 51s 229ms/step - loss: 0.4865 - accuracy: 0.8368 - val_loss: 0.7039 - val_accuracy: 0.7684
Epoch 7/20
225/225 [==============================] - 53s 236ms/step - loss: 0.4298 - accuracy: 0.8559 - val_loss: 0.7099 - val_

In [33]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 60s 221ms/step - loss: 0.7441 - accuracy: 0.7708 - val_loss: 0.6206 - val_accuracy: 0.7892
Epoch 2/20
263/263 [==============================] - 62s 235ms/step - loss: 0.6070 - accuracy: 0.8004 - val_loss: 0.6927 - val_accuracy: 0.7757
Epoch 3/20
263/263 [==============================] - 65s 249ms/step - loss: 0.5754 - accuracy: 0.8122 - val_loss: 0.7437 - val_accuracy: 0.7673
Epoch 4/20
263/263 [==============================] - 66s 250ms/step - loss: 0.5410 - accuracy: 0.8204 - val_loss: 0.6699 - val_accuracy: 0.7783
Epoch 5/20
263/263 [==============================] - 66s 251ms/step - loss: 0.5227 - accuracy: 0.8270 - val_loss: 0.6857 - val_accuracy: 0.7738
Epoch 6/20
263/263 [==============================] - 66s 251ms/step - loss: 0.5227 - accuracy: 0.8237 - val_loss: 0.6689 - val_accuracy: 0.7754
Epoch 7/20
263/263 [==============================] - 66s 252ms/step - loss: 0.5114 - accuracy: 0.8277 - val_loss: 0.6957 - val_

In [34]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 75s 242ms/step - loss: 0.7265 - accuracy: 0.7754 - val_loss: 0.7733 - val_accuracy: 0.7613
Epoch 2/20
300/300 [==============================] - 73s 243ms/step - loss: 0.6160 - accuracy: 0.7951 - val_loss: 0.6136 - val_accuracy: 0.7994
Epoch 3/20
300/300 [==============================] - 72s 242ms/step - loss: 0.5874 - accuracy: 0.8024 - val_loss: 0.6964 - val_accuracy: 0.7744
Epoch 4/20
300/300 [==============================] - 73s 244ms/step - loss: 0.5692 - accuracy: 0.8116 - val_loss: 0.6111 - val_accuracy: 0.7938
Epoch 5/20
300/300 [==============================] - 73s 243ms/step - loss: 0.5592 - accuracy: 0.8099 - val_loss: 0.6940 - val_accuracy: 0.7747
Epoch 6/20
300/300 [==============================] - 72s 241ms/step - loss: 0.5513 - accuracy: 0.8134 - val_loss: 0.7252 - val_accuracy: 0.7655
Epoch 7/20
300/300 [==============================] - 73s 243ms/step - loss: 0.5424 - accuracy: 0.8201 - val_loss: 0.6622 - val_

In [35]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 85s 245ms/step - loss: 0.7543 - accuracy: 0.7664 - val_loss: 0.6373 - val_accuracy: 0.7836
Epoch 2/20
338/338 [==============================] - 82s 244ms/step - loss: 0.6588 - accuracy: 0.7859 - val_loss: 0.6644 - val_accuracy: 0.7865
Epoch 3/20
338/338 [==============================] - 84s 247ms/step - loss: 0.6306 - accuracy: 0.7900 - val_loss: 0.6123 - val_accuracy: 0.7994
Epoch 4/20
338/338 [==============================] - 83s 245ms/step - loss: 0.6024 - accuracy: 0.8028 - val_loss: 0.5886 - val_accuracy: 0.8033
Epoch 5/20
338/338 [==============================] - 82s 242ms/step - loss: 0.5971 - accuracy: 0.8015 - val_loss: 0.7207 - val_accuracy: 0.7747
Epoch 6/20
338/338 [==============================] - 82s 243ms/step - loss: 0.5824 - accuracy: 0.8053 - val_loss: 0.6616 - val_accuracy: 0.7778
Epoch 7/20
338/338 [==============================] - 82s 242ms/step - loss: 0.5574 - accuracy: 0.8107 - val_loss: 0.6467 - val_

In [36]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 90s 234ms/step - loss: 0.7697 - accuracy: 0.7610 - val_loss: 0.6603 - val_accuracy: 0.8033
Epoch 2/20
375/375 [==============================] - 88s 234ms/step - loss: 0.6789 - accuracy: 0.7740 - val_loss: 0.6838 - val_accuracy: 0.7974
Epoch 3/20
375/375 [==============================] - 89s 237ms/step - loss: 0.6498 - accuracy: 0.7846 - val_loss: 0.6534 - val_accuracy: 0.7816
Epoch 4/20
375/375 [==============================] - 88s 234ms/step - loss: 0.6204 - accuracy: 0.7892 - val_loss: 0.7154 - val_accuracy: 0.7575
Epoch 5/20
375/375 [==============================] - 88s 235ms/step - loss: 0.6053 - accuracy: 0.7957 - val_loss: 0.6641 - val_accuracy: 0.7832
Epoch 6/20
375/375 [==============================] - 88s 235ms/step - loss: 0.5924 - accuracy: 0.8013 - val_loss: 0.7516 - val_accuracy: 0.7548
Epoch 7/20
375/375 [==============================] - 88s 236ms/step - loss: 0.6065 - accuracy: 0.7988 - val_loss: 0.6795 - val_

In [37]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 100s 237ms/step - loss: 0.7828 - accuracy: 0.7564 - val_loss: 0.6400 - val_accuracy: 0.7897
Epoch 2/20
413/413 [==============================] - 99s 239ms/step - loss: 0.6910 - accuracy: 0.7722 - val_loss: 0.6062 - val_accuracy: 0.8041
Epoch 3/20
413/413 [==============================] - 106s 258ms/step - loss: 0.6635 - accuracy: 0.7794 - val_loss: 0.6415 - val_accuracy: 0.7850
Epoch 4/20
413/413 [==============================] - 104s 251ms/step - loss: 0.6463 - accuracy: 0.7876 - val_loss: 0.7073 - val_accuracy: 0.7635
Epoch 5/20
413/413 [==============================] - 98s 237ms/step - loss: 0.6300 - accuracy: 0.7888 - val_loss: 0.6842 - val_accuracy: 0.7776
Epoch 6/20
413/413 [==============================] - 98s 237ms/step - loss: 0.6140 - accuracy: 0.7918 - val_loss: 0.6152 - val_accuracy: 0.7918
Epoch 7/20
413/413 [==============================] - 98s 237ms/step - loss: 0.6239 - accuracy: 0.7902 - val_loss: 0.7031 - 

In [38]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 106s 231ms/step - loss: 0.8064 - accuracy: 0.7396 - val_loss: 0.5779 - val_accuracy: 0.8087
Epoch 2/20
450/450 [==============================] - 104s 230ms/step - loss: 0.7240 - accuracy: 0.7550 - val_loss: 0.5612 - val_accuracy: 0.8123
Epoch 3/20
450/450 [==============================] - 104s 232ms/step - loss: 0.6861 - accuracy: 0.7678 - val_loss: 0.6130 - val_accuracy: 0.8037
Epoch 4/20
450/450 [==============================] - 104s 232ms/step - loss: 0.6739 - accuracy: 0.7697 - val_loss: 0.7523 - val_accuracy: 0.7648
Epoch 5/20
450/450 [==============================] - 104s 232ms/step - loss: 0.6585 - accuracy: 0.7752 - val_loss: 0.7037 - val_accuracy: 0.7696
Epoch 6/20
450/450 [==============================] - 104s 232ms/step - loss: 0.6354 - accuracy: 0.7855 - val_loss: 0.7018 - val_accuracy: 0.7699
Epoch 7/20
450/450 [==============================] - 104s 230ms/step - loss: 0.6455 - accuracy: 0.7828 - val_loss: 0.653

In [39]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 121s 243ms/step - loss: 0.8093 - accuracy: 0.7387 - val_loss: 0.6748 - val_accuracy: 0.7753
Epoch 2/20
488/488 [==============================] - 119s 245ms/step - loss: 0.7247 - accuracy: 0.7550 - val_loss: 0.5560 - val_accuracy: 0.8172
Epoch 3/20
488/488 [==============================] - 122s 249ms/step - loss: 0.7020 - accuracy: 0.7616 - val_loss: 0.5748 - val_accuracy: 0.8074
Epoch 4/20
488/488 [==============================] - 121s 248ms/step - loss: 0.6837 - accuracy: 0.7686 - val_loss: 0.6381 - val_accuracy: 0.7851
Epoch 5/20
488/488 [==============================] - 120s 246ms/step - loss: 0.6741 - accuracy: 0.7702 - val_loss: 0.6657 - val_accuracy: 0.7779
Epoch 6/20
488/488 [==============================] - 113s 232ms/step - loss: 0.6566 - accuracy: 0.7743 - val_loss: 0.6602 - val_accuracy: 0.7672
Epoch 7/20
488/488 [==============================] - 116s 238ms/step - loss: 0.6464 - accuracy: 0.7806 - val_loss: 0.632

In [40]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 121s 226ms/step - loss: 0.8423 - accuracy: 0.7309 - val_loss: 0.5547 - val_accuracy: 0.8227
Epoch 2/20
525/525 [==============================] - 118s 225ms/step - loss: 0.7253 - accuracy: 0.7528 - val_loss: 0.6454 - val_accuracy: 0.7959
Epoch 3/20
525/525 [==============================] - 119s 226ms/step - loss: 0.7023 - accuracy: 0.7641 - val_loss: 0.5648 - val_accuracy: 0.8121
Epoch 4/20
525/525 [==============================] - 119s 226ms/step - loss: 0.7056 - accuracy: 0.7624 - val_loss: 0.6414 - val_accuracy: 0.7837
Epoch 5/20
525/525 [==============================] - 119s 228ms/step - loss: 0.6891 - accuracy: 0.7661 - val_loss: 0.6405 - val_accuracy: 0.7836
Epoch 6/20
525/525 [==============================] - 120s 228ms/step - loss: 0.6688 - accuracy: 0.7696 - val_loss: 0.7199 - val_accuracy: 0.7703
Epoch 7/20
525/525 [==============================] - 119s 227ms/step - loss: 0.6512 - accuracy: 0.7807 - val_loss: 0.634

In [41]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 131s 228ms/step - loss: 0.8297 - accuracy: 0.7265 - val_loss: 0.7166 - val_accuracy: 0.7818
Epoch 2/20
563/563 [==============================] - 128s 228ms/step - loss: 0.7428 - accuracy: 0.7465 - val_loss: 0.6709 - val_accuracy: 0.7899
Epoch 3/20
563/563 [==============================] - 129s 228ms/step - loss: 0.7259 - accuracy: 0.7525 - val_loss: 0.6184 - val_accuracy: 0.7975
Epoch 4/20
563/563 [==============================] - 128s 228ms/step - loss: 0.7093 - accuracy: 0.7577 - val_loss: 0.6738 - val_accuracy: 0.7908
Epoch 5/20
563/563 [==============================] - 129s 229ms/step - loss: 0.7037 - accuracy: 0.7619 - val_loss: 0.6679 - val_accuracy: 0.7818
Epoch 6/20
563/563 [==============================] - 133s 237ms/step - loss: 0.6913 - accuracy: 0.7667 - val_loss: 0.6009 - val_accuracy: 0.8038
Epoch 7/20
563/563 [==============================] - 137s 243ms/step - loss: 0.6784 - accuracy: 0.7721 - val_loss: 0.612

In [42]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 143s 234ms/step - loss: 0.8405 - accuracy: 0.7243 - val_loss: 0.5771 - val_accuracy: 0.8121
Epoch 2/20
600/600 [==============================] - 140s 233ms/step - loss: 0.7595 - accuracy: 0.7394 - val_loss: 0.6109 - val_accuracy: 0.7928
Epoch 3/20
600/600 [==============================] - 139s 232ms/step - loss: 0.7486 - accuracy: 0.7438 - val_loss: 0.6268 - val_accuracy: 0.8002
Epoch 4/20
600/600 [==============================] - 141s 235ms/step - loss: 0.7244 - accuracy: 0.7547 - val_loss: 0.5817 - val_accuracy: 0.8092
Epoch 5/20
600/600 [==============================] - 141s 235ms/step - loss: 0.7136 - accuracy: 0.7563 - val_loss: 0.7286 - val_accuracy: 0.7735
Epoch 6/20
600/600 [==============================] - 141s 234ms/step - loss: 0.7104 - accuracy: 0.7582 - val_loss: 0.5851 - val_accuracy: 0.8066
Epoch 7/20
600/600 [==============================] - 141s 234ms/step - loss: 0.6875 - accuracy: 0.7647 - val_loss: 0.672

In [43]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 153s 236ms/step - loss: 0.8721 - accuracy: 0.7143 - val_loss: 0.5981 - val_accuracy: 0.8098
Epoch 2/20
638/638 [==============================] - 150s 236ms/step - loss: 0.7799 - accuracy: 0.7344 - val_loss: 0.6347 - val_accuracy: 0.7968
Epoch 3/20
638/638 [==============================] - 151s 236ms/step - loss: 0.7654 - accuracy: 0.7377 - val_loss: 0.6862 - val_accuracy: 0.7942
Epoch 4/20
638/638 [==============================] - 151s 236ms/step - loss: 0.7431 - accuracy: 0.7485 - val_loss: 0.6001 - val_accuracy: 0.7962
Epoch 5/20
638/638 [==============================] - 151s 236ms/step - loss: 0.7377 - accuracy: 0.7505 - val_loss: 0.6363 - val_accuracy: 0.7993
Epoch 6/20
638/638 [==============================] - 151s 237ms/step - loss: 0.7253 - accuracy: 0.7521 - val_loss: 0.6897 - val_accuracy: 0.7856
Epoch 7/20
638/638 [==============================] - 151s 237ms/step - loss: 0.7212 - accuracy: 0.7562 - val_loss: 0.731

In [ ]:
time.sleep(400)

In [44]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 159s 231ms/step - loss: 0.8619 - accuracy: 0.7141 - val_loss: 0.7185 - val_accuracy: 0.7923
Epoch 2/20
675/675 [==============================] - 156s 231ms/step - loss: 0.7839 - accuracy: 0.7332 - val_loss: 0.6008 - val_accuracy: 0.8109
Epoch 3/20
675/675 [==============================] - 157s 232ms/step - loss: 0.7700 - accuracy: 0.7413 - val_loss: 0.5511 - val_accuracy: 0.8132
Epoch 4/20
675/675 [==============================] - 157s 233ms/step - loss: 0.7540 - accuracy: 0.7407 - val_loss: 0.6094 - val_accuracy: 0.8115
Epoch 5/20
675/675 [==============================] - 158s 233ms/step - loss: 0.7386 - accuracy: 0.7502 - val_loss: 0.6017 - val_accuracy: 0.8138
Epoch 6/20
675/675 [==============================] - 157s 232ms/step - loss: 0.7322 - accuracy: 0.7533 - val_loss: 0.6386 - val_accuracy: 0.8047
Epoch 7/20
675/675 [==============================] - 160s 237ms/step - loss: 0.7316 - accuracy: 0.7477 - val_loss: 0.660

In [ ]:
time.sleep(400)

In [45]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 169s 233ms/step - loss: 0.8874 - accuracy: 0.7098 - val_loss: 0.6308 - val_accuracy: 0.7894
Epoch 2/20
713/713 [==============================] - 157s 220ms/step - loss: 0.7966 - accuracy: 0.7306 - val_loss: 0.6618 - val_accuracy: 0.7978
Epoch 3/20
713/713 [==============================] - 150s 211ms/step - loss: 0.7824 - accuracy: 0.7330 - val_loss: 0.7190 - val_accuracy: 0.7697
Epoch 4/20
713/713 [==============================] - 146s 205ms/step - loss: 0.7762 - accuracy: 0.7368 - val_loss: 0.6434 - val_accuracy: 0.8010
Epoch 5/20
713/713 [==============================] - 146s 205ms/step - loss: 0.7639 - accuracy: 0.7409 - val_loss: 0.6666 - val_accuracy: 0.7889
Epoch 6/20
713/713 [==============================] - 146s 204ms/step - loss: 0.7397 - accuracy: 0.7467 - val_loss: 0.6278 - val_accuracy: 0.7887
Epoch 7/20
713/713 [==============================] - 146s 205ms/step - loss: 0.7421 - accuracy: 0.7461 - val_loss: 0.589

In [ ]:
time.sleep(400)

In [46]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 198s 258ms/step - loss: 0.8938 - accuracy: 0.7051 - val_loss: 0.6586 - val_accuracy: 0.7929
Epoch 2/20
750/750 [==============================] - 181s 242ms/step - loss: 0.8251 - accuracy: 0.7158 - val_loss: 0.5964 - val_accuracy: 0.8217
Epoch 3/20
750/750 [==============================] - 180s 240ms/step - loss: 0.8059 - accuracy: 0.7244 - val_loss: 0.6203 - val_accuracy: 0.8130
Epoch 4/20
750/750 [==============================] - 182s 243ms/step - loss: 0.7782 - accuracy: 0.7343 - val_loss: 0.6580 - val_accuracy: 0.7935
Epoch 5/20
750/750 [==============================] - 183s 244ms/step - loss: 0.7730 - accuracy: 0.7384 - val_loss: 0.6521 - val_accuracy: 0.8054
Epoch 6/20
750/750 [==============================] - 206s 275ms/step - loss: 0.7588 - accuracy: 0.7431 - val_loss: 0.6130 - val_accuracy: 0.8116
Epoch 7/20
750/750 [==============================] - 210s 280ms/step - loss: 0.7766 - accuracy: 0.7327 - val_loss: 0.605

In [47]:
new_model_nc_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_nc_e1"

i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_nc_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_nc_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_nc_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_nc_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_nc_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_nc_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C2/cifar_model_c2_aug_nc_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to:

In [ ]:
loading = False

models_nc = []

if loading:
    for i in range(20):
        model_nc_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"+str(i)
        print(model_nc_dir)
        model =utils.My_model(dataset,True,model_nc_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_nc.append(model)
       

In [ ]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [ ]:
gc.collect()

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1